<a href="https://colab.research.google.com/github/marceloperrone/quind24/blob/main/Step_1_Imports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [3]:
import pandas as pd
import chardet
import numpy as np
import datetime
pd.set_option('display.max_columns', None)

df_dict= {}
df_list = {}

import_base = '/content/drive/MyDrive/FCB/Rural Reframe/Imports'

In [4]:
def adjust_dtypes(df):
  for column_name in df.columns:
    if column_name.endswith('PCT'):
      df[column_name] = df[column_name].apply(lambda x: round(x, 4))
      df[column_name] = df[column_name].astype(np.float64)
    if column_name.endswith('USD'):
        df[column_name] = df[column_name].apply(lambda x: round(x, 2))
        df[column_name] = df[column_name].astype(np.float64)
    if column_name.endswith(('_NUM', '_CAT', '_BIN')):
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
        df[column_name] = df[column_name].apply(lambda x: round(x) if pd.notna(x) else x)
        df[column_name] = df[column_name].astype(np.int64, errors='ignore') # Keep non-numeric as is
    if column_name.endswith('_LBL'):
      df[column_name] = df[column_name].apply(lambda x: str(x))
  return df

#Rural Continuum, Race, Ethnicity

*   CC-EST2022-ALLDATA-[ST-FIPS]: Annual County Resident Population Estimates by Age, Sex, Race, and Hispanic Origin: April 1, 2020 to July 1, 2022
*  File: 7/1/2022 County Characteristics Resident Population Estimates
* Source: U.S. Census Bureau, Population Division [link text](https://)
* Release date: June 2023

https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/counties/totals/

https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2023/

https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2022/cc-est2022-alldata.pdf

In [5]:
df = pd.read_csv(import_base+'/Population Counts/FIPS - Rural Continuum.csv',dtype={'FIPS': str,'STATE':str,'COUNTY':str})

# create totals for selected population counts
df["NH_WHITE_POP_NUM"] = df["NHWA_MALE"] +  df["NHWA_FEMALE"]
df["NH_BLACK_POP_NUM"] = df["NHBA_MALE"] +  df["NHBA_FEMALE"]
df["NH_AMIND_POP_NUM"] = df["NHIA_MALE"] +  df["NHIA_FEMALE"]
df["NH_ASIAN_POP_NUM"] = df["NHAA_MALE"] +  df["NHAA_FEMALE"]
df["NH_PACIFIC_POP_NUM"] = df["NHNA_MALE"] +  df["NHNA_FEMALE"]
df["NH_NHTOM_POP_NUM"] = df["NHTOM_MALE"] +  df["NHTOM_FEMALE"]
df["HISPANIC_POP_NUM"] = df["H_MALE"] +  df["H_FEMALE"]
df["NHWHITE_PCT"] = (df["NH_WHITE_POP_NUM"] / df["TOT_POP"])
df["NHBLACK_PCT"] = (df["NH_BLACK_POP_NUM"] / df["TOT_POP"])
df["HISPANIC_PCT"] = (df["HISPANIC_POP_NUM"] / df["TOT_POP"])


selected_columns = ['FIPS','STATE','COUNTY','STNAME','CTYNAME','RURAL CONTINUUM','TOT_POP']
race_ethnicity_columns = ['NH_WHITE_POP_NUM', 'NH_BLACK_POP_NUM', 'NH_AMIND_POP_NUM', 'NH_ASIAN_POP_NUM', 'NH_PACIFIC_POP_NUM','NH_NHTOM_POP_NUM','HISPANIC_POP_NUM',
                          'NHWHITE_PCT','NHBLACK_PCT', 'HISPANIC_PCT']

selected_columns.extend(race_ethnicity_columns)

df_popcounts = df[selected_columns].copy()
df_popcounts.rename(columns={'FIPS': 'Geo_FIPS',
                             'STATE':'State_FIPS',
                             'COUNTY':'County_FIPS',
                             'STNAME':'STATE_NAME',
                             'CTYNAME':'COUNTY_NAME',
                             'TOT_POP':'TOT_POP_NUM',
                             'RURAL CONTINUUM': 'R_CONTINUUM_CODE'}, inplace=True)

df_popcounts["Geo_FIPS"] = df_popcounts["Geo_FIPS"].astype(str)
df_popcounts['Geo_FIPS'] = df_popcounts['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_popcounts['Geo_FIPS'] = df_popcounts['Geo_FIPS'].str.replace(' ', '')
df_popcounts['R_CONTINUUM_CODE'] = df_popcounts['R_CONTINUUM_CODE'].astype(str)

df_popcounts = adjust_dtypes(df_popcounts)

df_popcounts.loc[df_popcounts['Geo_FIPS'] == "35013", 'COUNTY_NAME'] = 'Dona Ana County'
df_popcounts.sort_values(by="Geo_FIPS", inplace=True)

# add dataframe to master list
df_list['Rural Continuum with Race and Ethnicity'] = df_popcounts

# add dataframe columns to dictionary
df_dict['Geo_FIPS'] = 'Federal Information Processing Standard (FIPS) code for the geographic area'
df_dict['State_FIPS'] = 'FIPS code for the state'
df_dict['County_FIPS'] = 'FIPS code for the county'
df_dict['STATE_NAME'] = 'State name'
df_dict['COUNTY_NAME'] = 'County name'
df_dict['R_CONTINUUM_CODE'] = 'Rural Continuum Code'
df_dict['TOT_POP_NUM'] = 'Total population of the geographic area'
df_dict['NH_WHITE_POP_NUM'] = 'Number of non-Hispanic white people'
df_dict['NH_BLACK_POP_NUM'] = 'Number of non-Hispanic black people'
df_dict['NH_AMIND_POP_NUM'] = 'Number of American Indian and Alaska Native people'
df_dict['NH_ASIAN_POP_NUM'] = 'Number of Asian people'
df_dict['NH_PACIFIC_POP_NUM'] = 'Number of Native Hawaiian and Other Pacific Islander people'
df_dict['NH_NHTOM_POP_NUM'] = 'Number of people of two or more races'
df_dict['HISPANIC_POP_NUM'] = 'Number of Hispanic people'
df_dict['NHWHITE_PCT'] = 'Percentage of non-Hispanic white people'
df_dict['NHBLACK_PCT'] = 'Percentage of non-Hispanic black people'
df_dict['HISPANIC_PCT'] = 'Percentage of Hispanic people'

del df

# Economic Type
* https://www.ers.usda.gov/data-products/county-typology-codes/
* https://www.ers.usda.gov/webdocs/DataFiles/48652/2015CountyTypologyCodes.csv?v=419.9
* https://www.ers.usda.gov/data-products/county-typology-codes/documentation/

In [6]:
df_ecty = pd.read_csv(import_base+'/Economic Type/2015CountyTypologyCodes.csv', encoding="ISO-8859-1", low_memory=False)
df_ecty.rename(columns={"FIPStxt": "Geo_FIPS"}, inplace=True)
df_ecty.rename(columns={"Economic Types Type_2015_Update non-overlapping": "ECON_TYPE_CAT"}, inplace=True)
df_ecty.rename(columns={"Economic_Type_Label": "ECON_TYPE_LBL"}, inplace=True)
df_ecty.rename(columns={"Retirement_Dest_2015_Update": "ECON_TYPE_RETDEST_BIN"}, inplace=True)
df_ecty["Geo_FIPS"] = df_ecty["Geo_FIPS"].astype(str)
df_ecty['Geo_FIPS'] = df_ecty['Geo_FIPS'].str.replace(' ', '')
df_ecty['Geo_FIPS'] = df_ecty['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_ecty = df_ecty[['Geo_FIPS','ECON_TYPE_CAT','ECON_TYPE_LBL','ECON_TYPE_RETDEST_BIN']]

df_ecty = adjust_dtypes(df_ecty)

# add dataframe to master list
df_list['Economic Type'] = df_ecty

df_dict['ECON_TYPE_LBL'] = 'Name of the economic type classification by the BEA'
df_dict['ECON_TYPE_CAT'] = 'Economic classification (2015): 0-Nonspecialized, 1-Farming, 2-Mining, 3-Maufacturing, 4-Federal/State Government, 5-Recreation'
df_dict['ECON_TYPE_RETDEST_BIN'] = 'Flag 0/1 - Retirement destination'

# Economic Data

In [7]:
df_eda = pd.read_csv(import_base+'/Economic Data/CAINC4__ALL_AREAS_1969_2022.csv',
                     encoding="ISO-8859-1",
                     dtype={'GeoFIPS': str,'LineCode': str},
                     low_memory=False)


# 10 - PersInc_ - Personal income (thousands of dollars) - Consists of the income that persons receive in return for their provision of labor, land, and capital used in current production as well as other income, such as personal current transfer receipts. In the state and local personal income accounts the personal income of an area represents the income received by or on behalf of the persons residing in that area. It is calculated as the sum of wages and salaries, supplements to wages and salaries, proprietors' income with inventory valuation (IVA) and capital consumption adjustments (CCAdj), rental income of persons with capital consumption adjustment (CCAdj), personal dividend income, personal interest income, and personal current transfer receipts, less contributions for government social insurance plus the adjustment for residence.
# 11.0 - NonFarm_PersInc_ - Nonfarm personal income 1/ - Nonfarm personal income is personal income minus farm income.
# 12.0 - Farm income 2/ - Consists of wages and salaries, employer contributions for employee pension and insurance funds, and proprietors' income in the farm industry (NAICS subsectors 111-Crop Production and 112-Animal Production). Farm personal income comprises the net personal income of sole proprietors, partners, and hired laborers arising directly from the current production of agricultural commodities, both livestock and crops. It excludes corporate farm income.
# 20.0 - Population (persons) 3/ - The number of individuals (both civilian and military) who reside in a given area.
# 30.0 - Per capita personal income (dollars) 4/ - The personal income of a given area divided by the resident population of the area. See "personal income."
# 70 - Proprietors' income with inventory valuation and capital consumption adjustments is the current-production income (including income in kind) of sole proprietorships, partnerships, and tax-exempt cooperatives. Corporate directors' fees are included in proprietors' income. Proprietors' income includes the interest income received by financial partnerships and the net rental real estate income of those partnerships primarily engaged in the real estate business.
# 71 - Farm proprietors' income - Farm proprietors' income consists of the income that is received by the sole proprietorships and the partnerships that operate farms. It excludes the income that is received by corporate farms.
# 72 - Nonfarm proprietors' income - Nonfarm Proprietors' Income consists of the income that is received by nonfarm sole proprietorships and partnerships and the income that is received by tax-exempt cooperatives. The national estimates of nonfarm proprietors' income are primarily derived from income tax data. Because these data do not always reflect current production and because they are incomplete, the estimates also include four major adjustments--the inventory valuation adjustment, the capital consumption adjustment, the "misreporting" adjustment, and the adjustment for the net margins on owner-built housing. The inventory valuation adjustment offsets the effects of the gains and the losses that result from changes in the prices of products withdrawn from inventories; this adjustment for recent years has been small, but it is important to the definition of proprietors' income. The capital consumption adjustment changes the value of the consumption, or depreciation, of fixed capital from the historical-cost basis used in the source data to a replacement-cost basis. The "misreporting" adjustment adds an estimate of the income of sole proprietors and partnerships that is not reported on tax returns. The adjustment for the net margins on owner-built housing is an addition to the estimate for the construction industry. It is the imputed net income of individuals from the construction or renovation of their own dwellings. The source data necessary to prepare these adjustments are available only at the national level. Therefore, the national estimates of nonfarm proprietors' income that include the adjustments are allocated to states, and these state estimates are allocated to the counties, in proportion to tax return data that do not reflect the adjustments. In addition, the national estimates include adjustments made to reflect decreases in monetary and imputed income that result from damage to fixed capital and to inventories that is caused by disasters, such as hurricanes, floods, and earthquakes. These adjustments are attributed to states and counties on the basis of information from the Federal Emergency Management Agency.

line_codes = ['10', '11', '12', '20', '30', '7010','70','71','72']
column_prefixes = ['PERSINC', 'NONFARM_PERSINC', 'FARM_PERSINC', "TOT_POP", 'PERCAP_PERSINC','JOBS','PROP_INC','FARM_PROP_INC','NONFARM_PROP_INC']
selected_years = ['1992', '1997', '2002', '2007', '2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']

df_eda = df_eda[df_eda['LineCode'].isin(line_codes)]
df_eda = df_eda[['GeoFIPS', 'LineCode'] + selected_years]
df_eda.loc[:, 'GeoFIPS'] = df_eda['GeoFIPS'].str.replace('"', '').str.strip()
df_eda = df_eda[~df_eda['GeoFIPS'].str.endswith('000')]


In [8]:
# ONE WAY TO DO IT
def process_dataframe(df, line_codes, column_prefixes):
    results = []
    for line_code, prefix in zip(line_codes, column_prefixes):

        # Filter rows and clean up GeoFIPS
        df_temp = df[df['LineCode'].isin([line_code])]

        # Rename year columns with appropriate prefixes
        if prefix in ['PERSINC','NONFARM_PERSINC','FARM_PERSINC','PERCAP_PERSINC','PROP_INC','FARM_PROP_INC','NONFARM_PROP_INC']:
            suffix = 'USD'
        if prefix in ['TOT_POP','JOBS']:
            suffix = 'NUM'
        year_columns = {str(year): f'{prefix}_{year}_{suffix}' for year in selected_years}

        df_temp = df_temp.rename(columns=year_columns)
        df_temp = df_temp.drop(columns=['LineCode'])
        results.append(df_temp)

    # Merge all dataframes on 'GeoFIPS'
    df_result = results[0]

    for df_next in results[1:]:
        df_result = pd.merge(df_result, df_next, on="GeoFIPS")

    return df_result.sort_values(by="GeoFIPS")

# Process the dataframe
df_economic = process_dataframe(df_eda, line_codes, column_prefixes)

# destroy the variable that is no longer used
#del df_eda

for column in df_economic.columns:
  if column.startswith(('PERSINC', 'NONFARM_PERSINC', 'FARM_PERSINC','PERCAP_PERSINC','JOBS','PROP_INC','FARM_PROP_INC','NONFARM_PROP_INC')):
    df_economic[column].replace("N/A", "", inplace=True)
    df_economic[column].replace("(NA)", "", inplace=True)
    df_economic.loc[:, column] = pd.to_numeric(df_economic.loc[:, column])


df_economic.rename(columns={"GeoFIPS": "Geo_FIPS"}, inplace=True)

for year in selected_years:
    df_economic[f'FARM_PERSINC_{year}_PCT'] = df_economic[f'FARM_PERSINC_{year}_USD'] / df_economic[f'PERSINC_{year}_USD']


#df_economic.dropna(inplace=True)
df_economic = adjust_dtypes(df_economic)

# add dataframe to master list
df_list['Economic Data'] = df_economic

for year in selected_years:
    df_dict[f'PERSINC_{year}_USD'] = f'Personal income (thousands of dollars) {year}. Consists of the income that persons receive in return for their provision of labor, land, and capital used in current production as well as other income, such as personal current transfer receipts. In the state and local personal income accounts the personal income of an area represents the income received by or on behalf of the persons residing in that area. It is calculated as the sum of wages and salaries, supplements to wages and salaries, proprietors income with inventory valuation (IVA) and capital consumption adjustments (CCAdj), rental income of persons with capital consumption adjustment (CCAdj), personal dividend income, personal interest income, and personal current transfer receipts, less contributions for government social insurance plus the adjustment for residence.'
    df_dict[f'NONFARM_PERSINC_{year}_USD'] = f'Nonfarm personal income (thousands of dollars) {year}. Nonfarm personal income is personal income minus farm income.'
    df_dict[f'FARM_PERSINC_{year}_USD'] = f'Farm income for the year {year}. Consists of wages and salaries, employer contributions for employee pension and insurance funds, and proprietors income in the farm industry (NAICS subsectors 111-Crop Production and 112-Animal Production). Farm personal income comprises the net personal income of sole proprietors, partners, and hired laborers arising directly from the current production of agricultural commodities, both livestock and crops. It excludes corporate farm income.'
    df_dict[f'TOT_POP_{year}_NUM'] = f'The number of individuals (both civilian and military) who reside in a given area. {year}'
    df_dict[f'PERCAP_PERSINC_{year}_USD'] = f'Per capita personal income (dollars) {year} - The personal income of a given area divided by the resident population of the area. See "personal income."'
    df_dict[f'JOBS_{year}_NUM'] = f'Total Employment - A count of jobs, both full-time and part-time. It includes wage and salary jobs, sole proprietorships, and individual general partners, but not unpaid family workers nor volunteers. {year}'
    df_dict[f'FARM_PERSINC_{year}_PCT'] = f'Percentage of farm income for the year {year}'
    df_dict[f'PROP_INC_{year}_USD'] = f'Proprietors income with inventory valuation and capital consumption adjustments (dollars) {year}'
    df_dict[f'FARM_PROP_INC_{year}_USD'] = f'Farm proprietors income (dollars) {year}'
    df_dict[f'NONFARM_PROP_INC_{year}_USD'] = f'Nonfarm proprietors income (dollars) {year}'

# Unemployment
* https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/

In [9]:
df_unemp = pd.read_csv(import_base+'/Unemployment/Unemployment-Edited.csv')
df_unemp.rename(columns={"FIPS_Code": "Geo_FIPS"}, inplace=True)
df_unemp["Geo_FIPS"] = df_unemp["Geo_FIPS"].astype(str)
df_unemp['Geo_FIPS'] = df_unemp['Geo_FIPS'].str.replace(' ', '')
df_unemp['Geo_FIPS'] = df_unemp['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_unemp = df_unemp.drop(columns=['State','Area_Name'])

rename_dict = {}
for year in range(2000, 2023):
  df_unemp[f'Unemployment_rate_{year}'] = df_unemp[f'Unemployment_rate_{year}'].apply(lambda x: round(x/100,4))
  rename_dict[f'Unemployment_rate_{year}'] = f'UNEMP_RATE_{year}_PCT'

df_unemp.rename(columns=rename_dict, inplace=True)

df_unemp = adjust_dtypes(df_unemp)

# add dataframe to master list
df_list['Unemployment'] = df_unemp

for year in range(2000, 2023):
  df_dict[f'UNEMP_RATE_{year}_PCT'] = f'Annual unemployment rate for the year {year}'



# Education
* Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970–2021
* Note: For 1970 and 1980, the share of adults with less than high school includes those who had not completed the 12th grade. In 1990, 2000, and estimates from the 5-year period American Conmunity Survey, the share includes those who did not receive a high school diploma or its equivalent (such as a GED).
* https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/

In [10]:
df_education = pd.read_csv(import_base+'/Education/Education-Edited.csv')
df_education["GeoFIPS"] = df_education["GeoFIPS"].astype(str)
df_education['GeoFIPS'] = df_education['GeoFIPS'].str.replace(' ', '')
df_education['GeoFIPS'] = df_education['GeoFIPS'].apply(lambda x: x.zfill(5))
df_education = df_education.drop(columns=['State','Area name'])

df_education_renames =  {'GeoFIPS': 'Geo_FIPS',
                         'Pct_NoHighSchool_2021':'EDU_NOHIGH_2021_PCT',
                         'Pct_HighSchoolOnly_2021':'EDU_HIGHONLY_2021_PCT',
                         'Pct_CollegeAssociate_2021':'EDU_COLLEGE_2021_PCT',
                         'Pct_BachOrHigher_2021':'EDU_BACH_2021_PCT'}

df_education.rename(columns=df_education_renames, inplace=True)

df_education['EDU_NOHIGH_2021_PCT'] = df_education['EDU_NOHIGH_2021_PCT'].apply(lambda x: round(x/100,4))
df_education['EDU_HIGHONLY_2021_PCT'] = df_education['EDU_HIGHONLY_2021_PCT'].apply(lambda x: round(x/100,4))
df_education['EDU_COLLEGE_2021_PCT'] = df_education['EDU_COLLEGE_2021_PCT'].apply(lambda x: round(x/100,4))
df_education['EDU_BACH_2021_PCT'] = df_education['EDU_BACH_2021_PCT'].apply(lambda x: round(x/100,4))

df_education = adjust_dtypes(df_education)

# add dataframe to master list
df_list['Education'] = df_education

df_dict['EDU_NOHIGH_2021_PCT'] = 'Percentage of population 25 years and over with less than a high school diploma in 2021'
df_dict['EDU_HIGHONLY_2021_PCT'] = 'Percentage of population 25 years and over with a high school diploma in 2021'
df_dict['EDU_COLLEGE_2021_PCT'] = 'Percentage of population 25 years and over with some college, no degree in 2021'
df_dict['EDU_BACH_2021_PCT'] = 'Percentage of population 25 years and over with a Bachelor degree in 2021'

# Election

In [11]:
df_election = pd.read_csv(import_base+'/Election/Election_2000-2020.csv', dtype={'county_fips': str})
df_election.rename(columns={"county_fips": "Geo_FIPS"}, inplace=True)

df_election["Geo_FIPS"] = df_election["Geo_FIPS"].astype(str)
df_election['Geo_FIPS'] = df_election['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_election['Geo_FIPS'] = df_election['Geo_FIPS'].str.replace(' ', '')

df_election = df_election[df_election['mode'] == 'TOTAL']
df_election = df_election[df_election["party"].isin(["DEMOCRAT", "REPUBLICAN"])]
df_election = df_election[df_election['county_name'] != 'STATEWIDE WRITEIN']
df_election = df_election[df_election['county_name'] != 'MAINE UOCAVA']
df_election = df_election[df_election['county_name'] != 'FEDERAL PRECINCT']

dfs_temp = {}

for year in [2000,2004,2008,2012,2016,2020]:
    df_temp_d = df_election[df_election['year'] == year]
    df_temp_d = df_temp_d[df_temp_d['party'] == 'DEMOCRAT']
    df_temp_d.loc[:,'ELE_DEMOCRAT_' + str(year)+ '_PCT'] = df_temp_d['candidatevotes']/df_temp_d['totalvotes']
    df_temp_d = df_temp_d[["Geo_FIPS",'ELE_DEMOCRAT_' + str(year)+ '_PCT']]

    df_temp_r = df_election[df_election['year'] == year]
    df_temp_r = df_temp_r[df_temp_r['party'] == 'REPUBLICAN']
    df_temp_r.loc[:,"ELE_REPUBLICAN_" + str(year)+ '_PCT'] = df_temp_r['candidatevotes']/df_temp_r['totalvotes']
    df_temp_r = df_temp_r[['Geo_FIPS','ELE_REPUBLICAN_' + str(year)+ '_PCT']]

    dfs_temp[str(year)] = pd.merge(df_temp_d, df_temp_r, on="Geo_FIPS", how="outer")


df_temp_all = pd.merge(dfs_temp['2000'], dfs_temp['2004'], on="Geo_FIPS", how="outer")
df_temp_all = pd.merge(df_temp_all, dfs_temp['2008'], on="Geo_FIPS", how="outer")
df_temp_all = pd.merge(df_temp_all, dfs_temp['2012'], on="Geo_FIPS", how="outer")
df_temp_all = pd.merge(df_temp_all, dfs_temp['2016'], on="Geo_FIPS", how="outer")
df_temp_all = pd.merge(df_temp_all, dfs_temp['2020'], on="Geo_FIPS", how="outer")

df_elections = df_temp_all.copy()

df_elections = adjust_dtypes(df_elections)

# add dataframe to master list
df_list['Elections'] = df_elections

for year in [2000,2004,2008,2012,2016,2020]:
    for party in ['DEMOCRAT','REPUBLICAN']:
        df_dict[f'ELE_{party}_{year}_PCT'] = f'Percentage of votes for the {party} candidate in {year}'


# Poverty

In [12]:
df_poverty = pd.read_csv(import_base+'/Poverty/PovertyEstimates-Edited.csv')
df_poverty.rename(columns={"FIPS_Code": "Geo_FIPS"}, inplace=True)
df_poverty["Geo_FIPS"] = df_poverty["Geo_FIPS"].astype(str)
df_poverty['Geo_FIPS'] = df_poverty['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_poverty['Geo_FIPS'] = df_poverty['Geo_FIPS'].str.replace(' ', '')
df_poverty = df_poverty.drop(columns=['PCTPOV517_2021','Area_name'])

df_poverty_renames =  {'POVALL_2021':'POV_POVERTYPOP_2021_NUM',
                       'PCTPOVALL_2021':'POV_POVERTYPOP_2021_PCT',
                       'PCTPOV017_2021':'POV_POVERTYPOP17YR_2021_PCT',
                       'MEDHHINC_2021':'POV_MEDHHINC_2021_USD'}

df_poverty.rename(columns=df_poverty_renames, inplace=True)

df_poverty['POV_POVERTYPOP_2021_PCT'] = df_poverty['POV_POVERTYPOP_2021_PCT'].apply(lambda x: round(x/100,4))
df_poverty['POV_POVERTYPOP17YR_2021_PCT'] = df_poverty['POV_POVERTYPOP17YR_2021_PCT'].apply(lambda x: round(x/100,4))

df_poverty = adjust_dtypes(df_poverty)

# add dataframe to master list
df_list['Poverty'] = df_poverty

# add metadata to dictionary
df_dict['POV_POVERTYPOP_2021_NUM'] = 'Estimate of people of all ages in poverty 2021'
df_dict['POV_POVERTYPOP_2021_PCT'] = 'Estimated percent of people of all ages in poverty 2021'
df_dict['POV_POVERTYPOP17YR_2021_PCT'] = 'Estimate of people age 0-17 in poverty 2021'
df_dict['POV_MEDHHINC_2021_USD'] = 'Estimate of median household income 2021'

# Food Access
* Feeding America
* https://feedingamerica.az1.qualtrics.com/jfe/form/SV_5tJt5m9K62hRC6N
* https://www.feedingamerica.org/research/map-the-meal-gap/by-county

In [13]:
df_food = pd.read_csv(import_base+'/Food Access/Food_Insecurity.csv')
df_food = df_food.drop(columns=['State','County'])
df_food["GeoFIPS"] = df_food["GeoFIPS"].astype(str)
df_food['GeoFIPS'] = df_food['GeoFIPS'].str.replace(' ', '')
df_food['GeoFIPS'] = df_food['GeoFIPS'].apply(lambda x: x.zfill(5))

df_food['Food_Insecurity_Rate_2021'] = df_food['Food_Insecurity_Rate_2021'].str.replace('%', '')
df_food['Food_Insecurity_Rate_2021'] = pd.to_numeric(df_food['Food_Insecurity_Rate_2021'])
df_food['Food_Insecurity_Rate_2021'] = df_food['Food_Insecurity_Rate_2021'].apply(lambda x: round(x/100,4))

df_food['CHD_Food_Insecurity_Rate_2021'] = df_food['CHD_Food_Insecurity_Rate_2021'].str.replace('%', '')
df_food['CHD_Food_Insecurity_Rate_2021'] = pd.to_numeric(df_food['CHD_Food_Insecurity_Rate_2021'])
df_food['CHD_Food_Insecurity_Rate_2021'] = df_food['CHD_Food_Insecurity_Rate_2021'].apply(lambda x: round(x/100,4))

df_food['CHD_Food_Insecurity_Number'] = df_food['CHD_Food_Insecurity_Number'].str.replace(',', '')

df_food_renames =  {'GeoFIPS':'Geo_FIPS',
                    'Food_Insecurity_Rate_2021':'FOOD_INSRATE_2021_PCT',
                    'CHD_Food_Insecurity_Rate_2021':'FOOD_CHDINSRATE_2021_PCT',
                    'CHD_Food_Insecurity_Number':'FOOD_CHDINS_2021_NUM'}

df_food.rename(columns=df_food_renames, inplace=True)

df_food = adjust_dtypes(df_food)

# add dataframe to master list
df_list['Food Access'] = df_food

# add metadata to dictionary
df_dict['FOOD_INSRATE_2021_PCT'] = 'Food Insecurity Rate 2021'
df_dict['FOOD_CHDINSRATE_2021_PCT'] = 'Child Food Insecurity Rate 2021'
df_dict['FOOD_CHDINS_2021_NUM'] = 'Number Food Insecurity Children in 2021'

# COVID-19 Vax
* https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh/about_data

In [14]:
df_covid = pd.read_csv(import_base+'/Health - COVID-19 Vaccination/COVID-19-Short.csv')
df_covid.rename(columns={"FIPS": "Geo_FIPS"}, inplace=True)
df_covid["Geo_FIPS"] = df_covid["Geo_FIPS"].astype(str)
df_covid['Geo_FIPS'] = df_covid['Geo_FIPS'].str.replace(' ', '')
df_covid['Geo_FIPS'] = df_covid['Geo_FIPS'].apply(lambda x: x.zfill(5))

df_covid.rename(columns={'Series_Complete_Pop_Pct': 'CVD19_PRIMARY_PCT',
                         'Booster_Doses_Vax_Pct': 'CVD19_BOOSTER_PCT'
                         }, inplace=True)

df_covid = df_covid[df_covid['Geo_FIPS'] != '00UNK']
df_covid = df_covid.drop(columns=['Recip_County','Recip_State','Administered_Dose1_Pop_Pct','Completeness_pct'])

df_covid['CVD19_PRIMARY_PCT'] = round(df_covid['CVD19_PRIMARY_PCT']/100,4)
df_covid['CVD19_BOOSTER_PCT'] = round(df_covid['CVD19_BOOSTER_PCT']/100,4)

df_covid = adjust_dtypes(df_covid)

# add dataframe to master list
df_list['COVID-19 Vaccination'] = df_covid

df_dict['CVD19_PRIMARY_PCT'] = 'Represents the proportion of people with a completed primary series whose Federal Information Processing Standards (FIPS) code is reported and matches a valid county FIPS code in the jurisdiction.'
df_dict['CVD19_BOOSTER_PCT'] = 'Percent of people who completed a primary series and have received a booster (or additional) dose.'


# Health Measures


In [15]:
df_health = pd.read_csv(import_base+'/Health Measures/swc5-untb_version_145.csv')
df_health = df_health[df_health["year"] == 2021]
df_health = df_health[df_health["datavaluetypeid"] == "AgeAdjPrv"]
df_health = df_health.drop(columns=['datasource','year','totalpopulation','stateabbr','statedesc','locationname','category','categoryid','short_question_text','measure','geolocation','data_value_unit','data_value_type','datavaluetypeid','data_value_footnote_symbol','data_value_footnote','latitude','longitude','low_confidence_limit','high_confidence_limit'])
df_health.rename(columns={"locationid": "Geo_FIPS"}, inplace=True)
df_health["Geo_FIPS"] = df_health["Geo_FIPS"].astype(str)
df_health['Geo_FIPS'] = df_health['Geo_FIPS'].str.replace(' ', '')
df_health['Geo_FIPS'] = df_health['Geo_FIPS'].apply(lambda x: x.zfill(5))

measures = ["DISABILITY","DEPRESSION","DIABETES","OBESITY","GHLTH","MHLTH","PHLTH","ACCESS2","BINGE","CSMOKING"]
df_results = []

for measure in measures:
    df_temp = df_health[df_health["measureid"] == measure]
    df_temp = df_temp.drop(columns=['measureid'])
    df_temp.rename(columns={"data_value": f"{measure}_2021_PCT"}, inplace=True)
    df_results.append(df_temp)

df_healthmeasures = df_results[0]

for df in df_results[1:]:
    df_healthmeasures = pd.merge(df_healthmeasures, df, on="Geo_FIPS")

for measure in measures:
    df_healthmeasures[f"{measure}_2021_PCT"] = df_healthmeasures[f"{measure}_2021_PCT"]/100

df_healthmeasures = adjust_dtypes(df_healthmeasures)

# add dataframe to master list
df_list['Health Measures'] = df_healthmeasures

df_dict['DISABILITY_2021_PCT'] = 'Any disability among adults aged >=18 years'
df_dict['DEPRESSION_2021_PCT'] = 'Depression among adults aged >=18 years'
df_dict['DIABETES_2021_PCT'] = 'Diagnosed diabetes among adults aged >=18 years'
df_dict['OBESITY_2021_PCT'] = 'Obesity among adults aged >=18 years'
df_dict['GHLTH_2021_PCT'] = 'Fair or poor self-rated health status among adults aged >=18 years'
df_dict['MHLTH_2021_PCT'] = 'Mental health not good for >=14 days among adults aged >=18 years'
df_dict['PHLTH_2021_PCT'] = 'Physical health not good for >=14 days among adults aged >=18 years'
df_dict['ACCESS2_2021_PCT'] = 'Current lack of health insurance among adults aged 18-64 years'
df_dict['BINGE_2021_PCT'] = 'Binge drinking among adults aged >=18 years'
df_dict['CSMOKING_2021_PCT'] = 'Current smoking among adults aged >=18 years'

# Population Changes
* Population estimates for the U.S., States, and counties, 2020–22
* Sources: U.S. Department of Commerce, Bureau of the Census, Population Estimates Program 2022 and 2020 Decennial Census.
* For definitions of rural classifications, see the USDA, Economic Research Service webpage 'Rural Classifications' in the 'Rural Economy & Population' topic. Note that counties not existing when rural classifications were determined will not have values for those fields. Variable descriptions (column names) are found in the second tab in this workbook.
* This table was prepared by USDA, Economic Research Service. Data as of June 16, 2023. Contact: Austin Sanders, austin.sanders@usda.gov


In [16]:
#PopulationEstimates.csv
df_popchanges = pd.read_csv(import_base+'/Population Changes/PopulationEstimates.csv', dtype={'FIPStxt': str, 'Economic_typology_2015': str})
df_popchanges.rename(columns={"FIPStxt": "Geo_FIPS"}, inplace=True)
df_popchanges["Geo_FIPS"] = df_popchanges["Geo_FIPS"].astype(str)
df_popchanges['Geo_FIPS'] = df_popchanges['Geo_FIPS'].str.replace(' ', '')
df_popchanges['Geo_FIPS'] = df_popchanges['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_popchanges.drop(columns=['Rural_Urban_Continuum_Code_2013','Rural_Urban_Continuum_Code_2003','Urban_Influence_2003','State','Area_Name','CENSUS_2020_POP','ESTIMATES_BASE_2020','RESIDUAL_2020','RESIDUAL_2021','RESIDUAL_2022','GQ_ESTIMATES_BASE_2020','GQ_ESTIMATES_2020','GQ_ESTIMATES_2021','GQ_ESTIMATES_2022'],inplace=True)

# add dataframe to master list
df_list['Population Changes'] = df_popchanges

selected_variables = ['POP_ESTIMATE','N_POP_CHG','BIRTHS','DEATHS','NATURAL_CHG','INTERNATIONAL_MIG','DOMESTIC_MIG','NET_MIG',
                      'R_DOMESTIC_MIG','R_INTERNATIONAL_MIG','R_NATURAL_CHG','R_BIRTH','R_DEATH','R_NET_MIG']
selected_years = ['2020','2021','2022']

df_popchanges_renames={'Urban_Influence_2013': 'PCH_URBAN_INFLUENCE_2013_CAT',
                       'Economic_typology_2015': 'PCH_ECO_TYPO_2015_CAT'}

for year in selected_years:
    df_popchanges_renames[f'POP_ESTIMATE_{year}'] = f'PCH_POP_{year}_NUM'
    df_popchanges_renames[f'N_POP_CHG_{year}'] = f'PCH_POPCHANGE_{year}_NUM'
    df_popchanges_renames[f'BIRTHS_{year}'] = f'PCH_BIRTHS_{year}_NUM'
    df_popchanges_renames[f'DEATHS_{year}'] = f'PCH_DEATHS_{year}_NUM'
    df_popchanges_renames[f'NATURAL_CHG_{year}'] = f'PCH_NATURALCHG_{year}_NUM'
    df_popchanges_renames[f'INTERNATIONAL_MIG_{year}'] = f'PCH_INTLMIG_{year}_NUM'
    df_popchanges_renames[f'DOMESTIC_MIG_{year}'] = f'PCH_DOMESTICMIG_{year}_NUM'
    df_popchanges_renames[f'NET_MIG_{year}'] = f'PCH_NETMIG_{year}_NUM'
    if year != '2020':
        df_popchanges_renames[f'R_DOMESTIC_MIG_{year}'] = f'PCH_R_DOMESTICMIG_{year}_PCT'
        df_popchanges_renames[f'R_INTERNATIONAL_MIG_{year}'] = f'PCH_R_INTLMIG_{year}_PCT'
        df_popchanges_renames[f'R_NATURAL_CHG_{year}'] = f'PCH_R_NATURALCHG_{year}_PCT'
        df_popchanges_renames[f'R_BIRTH_{year}'] = f'PCH_R_BIRTH_{year}_PCT'
        df_popchanges_renames[f'R_DEATH_{year}'] = f'PCH_R_DEATH_{year}_PCT'
        df_popchanges_renames[f'R_NET_MIG_{year}'] = f'PCH_R_NETMIG_{year}_PCT'

df_popchanges.rename(columns=df_popchanges_renames, inplace=True)


for col in [col for col in df_popchanges.columns if col.endswith('_PCT')]:
    df_popchanges[col] = df_popchanges[col].apply(lambda x: round(x/100,4))


for col in [col for col in df_popchanges.columns if col.endswith('_NUM')]:
    df_popchanges[col] = df_popchanges[col].astype(str).apply(lambda x: x.replace(',',''))
    df_popchanges[col] = pd.to_numeric(df_popchanges[col], errors='coerce')


df_dict['PCH_URBAN_INFLUENCE_2013_CAT'] = 'Urban Influence Code, 2013'
df_dict['PCH_ECO_TYPO_2015_CAT'] = 'County economic types, 2015 edition. Non-overlapping economic-dependence county indicator. 0=Nonspecialized 1=Farm-dependent 2=Mining-dependent 3=Manufacturing-dependent 4=Federal/State government-dependent 5=Recreation'
for year in selected_years:
    df_dict[f'PCH_POP_{year}_NUM'] = f'7/1/{year} resident total population estimate'
    df_dict[f'PCH_POPCHANGE_{year}_NUM'] = f'Numeric change in resident total population for {year}'
    df_dict[f'PCH_BIRTHS_{year}_NUM'] = f'Births in {year}'
    df_dict[f'PCH_DEATHS_{year}_NUM'] = f'Deaths in {year}'
    df_dict[f'PCH_NATURALCHG_{year}_NUM'] = f'Natural change in population number in {year}'
    df_dict[f'PCH_INTLMIG_{year}_NUM'] = f'International migration in {year}'
    df_dict[f'PCH_DOMESTICMIG_{year}_NUM'] = f'Domestic migration in {year}'
    df_dict[f'PCH_NETMIG_{year}_NUM'] = f'Net migration in {year}'
    if year != '2020':
        df_dict[f'PCH_R_DOMESTICMIG_{year}_PCT'] = f'Domestic migration rate'
        df_dict[f'PCH_R_INTLMIG_{year}_PCT'] = f'International migration rate'
        df_dict[f'PCH_R_NATURALCHG_{year}_PCT'] = f'Natural change rage'
        df_dict[f'PCH_R_BIRTH_{year}_PCT'] = f'Birth rate'
        df_dict[f'PCH_R_DEATH_{year}_PCT'] = f'Death rate'
        df_dict[f'PCH_R_NETMIG_{year}_PCT'] = f'Net migration rate'


# GDP
* https://www.bea.gov/data/gdp/gdp-county-metro-and-other-areas
* https://www.bea.gov/sites/default/files/2023-12/lagdp1223.pdf
* https://apps.bea.gov/itable/?ReqID=70&step=1&_gl=1*tpdujr*_ga*MjAzNjE2MzA4NS4xNzE4MDMzMTQ1*_ga_J4698JNNFT*MTcxODcyMDYxNy42LjEuMTcxODcyMDc2My40OS4wLjA.

In [17]:

df_gdp = pd.read_csv(import_base+'/GDP/USBEA_GDP-Edited-w-GeoFIPS.csv', dtype={'GEOFIPS': str})
df_gdp.rename(columns={'GEOFIPS': 'Geo_FIPS',
                       'GDP_2019': 'GDP_2019_USD',
                       'GDP_2020': 'GDP_2020_USD',
                       'GDP_2021': 'GDP_2021_USD',
                       'GDP_2022': 'GDP_2022_USD',
                       'GDPGrowth_2020': 'GDP_GROWTH_2020_PCT',
                       'GDPGrowth_2021': 'GDP_GROWTH_2021_PCT',
                       'GDPGrowth_2022': 'GDP_GROWTH_2022_PCT'
                       }, inplace=True)
df_gdp['Geo_FIPS'] = df_gdp['Geo_FIPS'].str.replace(' ', '')
df_gdp["Geo_FIPS"] = df_gdp["Geo_FIPS"].astype(str)
df_gdp['Geo_FIPS'] = df_gdp['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_gdp.drop(columns=['Unnamed: 0','County_Name','STATE'],inplace=True)

df_gdp = df_gdp[df_gdp['Geo_FIPS'] != '00nan']

for column in df_gdp.columns:
    if column.startswith(("GDP_2")):
        df_gdp[column] = df_gdp[column].str.replace(',', '')
        df_gdp[column] = pd.to_numeric(df_gdp[column])
    if column.startswith(("GDP_Growth_")):
        df_gdp[column] = pd.to_numeric(df_gdp[column], errors='coerce')
        df_gdp[column] = round(df_gdp[column]/100,4)
# add dataframe to master list
df_list['GDP'] = df_gdp

df_dict['GDP_2019_USD'] = '2019 - Real GDP by county was prepared in chained (2012) dollars. Real GDP by county is an inflation-adjusted measure of each county based on national prices. These measures are important when making comparisons over time and when calculating growth.'
df_dict['GDP_2020_USD'] = '2020 - GDP is computed as the sum of compensation of employees (COMP), taxes on production and imports (TOPI) less subsidies (SUB), and gross operating surplus (GOS).Real GDP by county was prepared in chained (2012) dollars. Real GDP by county is an inflation-adjusted measure of each county based on national prices. These measures are important when making comparisons over time and when calculating growth.'
df_dict['GDP_2021_USD'] = '2021 - GDP is computed as the sum of compensation of employees (COMP), taxes on production and imports (TOPI) less subsidies (SUB), and gross operating surplus (GOS).Real GDP by county was prepared in chained (2012) dollars. Real GDP by county is an inflation-adjusted measure of each county based on national prices. These measures are important when making comparisons over time and when calculating growth.'
df_dict['GDP_2022_USD'] = '2022 - GDP is computed as the sum of compensation of employees (COMP), taxes on production and imports (TOPI) less subsidies (SUB), and gross operating surplus (GOS).Real GDP by county was prepared in chained (2012) dollars. Real GDP by county is an inflation-adjusted measure of each county based on national prices. These measures are important when making comparisons over time and when calculating growth.'
df_dict['GDP_GROWTH_2020_PCT'] = '2020 GDP Growth percentage over 2019'
df_dict['GDP_GROWTH_2021_PCT'] = '2021 GDP Growth percentage over 2020'
df_dict['GDP_GROWTH_2022_PCT'] = '2022 GDP Growth percentage over 2021'


# GINI
* ID: ACSDT5Y2022.B19083
* Title: Gini Index of Income Inequality

In [18]:
df_gini = pd.read_csv(import_base+'/GINI/ACSDT5Y2022.B19083-Data-Clean.csv', dtype={'GEO_ID': str})
df_gini['GEO_ID'] = df_gini['GEO_ID'].str[-5:]
df_gini.rename(columns={'B19083_001E_GINI':'GINI_2022_IND'}, inplace=True)
df_gini.rename(columns={"GEO_ID": "Geo_FIPS"}, inplace=True)
df_gini.drop(columns=['NAME'],inplace=True)


# add dataframe to master list
df_list['GINI'] = df_gini

df_dict['GINI_2022_IND'] = '2022 GINI Index'

#Health Insurance
* ID: ACSDT5Y2022.B27010
* Title: Types of Health Insurance Coverage by Age
* Source: U.S. Census Bureau, 2018-2022 American Community Survey 5-Year Estimates


In [19]:
df_hins = pd.read_csv(import_base+'/Health Insurance/ACSDT5Y2022.B27010_2024-06-16T205816/ACSDT5Y2022.B27010-Data.csv', dtype={'GEO_ID': str})
df_hins['GEO_ID'] = df_hins['GEO_ID'].str[-5:]
df_hins = df_hins.drop(index=0)

df_hins.rename(columns={'GEO_ID':'Geo_FIPS',
                        'B27010_001E':'HINS_TOTPOP_NUM',           # Total
                        'B27010_002E':'HINS_U19YRS_NUM',             # Under 19 years
                        'B27010_003E':'HINS_U19YRS_1HI_NUM',         # Under 19 years - With one type of health insurance coverage
                        'B27010_010E':'HINS_U19YRS_2PLUS_NUM',       # Under 19 years - With two or more types of health insurance coverage
                        'B27010_017E':'HINS_U19YRS_NHI_NUM',         # Under 19 years - No health insurance coverage
                        'B27010_018E':'HINS_1934YRS_NUM',            # 19 to 34 years
                        'B27010_019E':'HINS_1934YRS_1HI_NUM',        # 19 to 34 years - With one type of health insurance coverage
                        'B27010_026E':'HINS_1934YRS_2PLUS_NUM',      # 19 to 34 years - With two or more types of health insurance coverage
                        'B27010_033E':'HINS_1934YRS_NHI_NUM',        # 19 to 34 years - No health insurance coverage
                        'B27010_034E':'HINS_3564YRS_NUM',            # 35 to 64 years
                        'B27010_035E':'HINS_3564YRS_1HI_NUM',        # 35 to 64 years - With one type of health insurance coverage
                        'B27010_042E':'HINS_3564YRS_2PLUS_NUM',      # 35 to 64 years - With two or more types of health insurance coverage
                        'B27010_050E':'HINS_3564YRS_NHI_NUM',        # 35 to 64 years - No health insurance coverage
                        'B27010_051E':'HINS_65UPYRS_NUM',            # 65 years and over
                        'B27010_052E':'HINS_65UPYRS_1HI_NUM',        # 65 years and over - With one type of health insurance coverage
                        'B27010_058E':'HINS_65UPYRS_2PLUS_NUM',      # 65 years and over - With two or more types of health insurance coverage
                        'B27010_066E':'HINS_65UPYRS_NHI_NUM'         # 65 years and over - No health insurance coverage
                        }, inplace=True)

# PCT OF TOTAL POPULATION THAT IS NOT INSURED
df_hins['HINS_NOTINSURED_PCT'] = (df_hins["HINS_U19YRS_NHI_NUM"].astype(int) + df_hins["HINS_1934YRS_NHI_NUM"].astype(int) + df_hins["HINS_3564YRS_NHI_NUM"].astype(int)+ df_hins["HINS_65UPYRS_NHI_NUM"].astype(int)) / df_hins["HINS_TOTPOP_NUM"].astype(int)

# PCT OF POPULATION <19YO THAT IS NOT INSURED
df_hins['HINS_NOTINSURED_19U_PCT'] = df_hins["HINS_U19YRS_NHI_NUM"].astype(int) / df_hins["HINS_U19YRS_NUM"].astype(int)

# PCT OF POPULATION 19-34YO THAT IS NOT INSURED
df_hins['HINS_NOTINSURED_1934YRS_PCT'] = df_hins["HINS_1934YRS_NHI_NUM"].astype(int) / df_hins["HINS_1934YRS_NUM"].astype(int)

# PCT OF POPULATION 35-64YO THAT IS NOT INSURED
df_hins['HINS_NOTINSURED_3564YRS_PCT'] = df_hins["HINS_3564YRS_NHI_NUM"].astype(int) / df_hins["HINS_3564YRS_NUM"].astype(int)

# PCT OF POPULATION 65YOUP THAT IS NOT INSURED
df_hins['HINS_NOTINSURED_65UPYRS_PCT'] = df_hins["HINS_65UPYRS_NHI_NUM"].astype(int) / df_hins["HINS_65UPYRS_NUM"].astype(int)

df_hins = df_hins[['Geo_FIPS',
                   'HINS_TOTPOP_NUM','HINS_U19YRS_NUM','HINS_U19YRS_1HI_NUM','HINS_U19YRS_2PLUS_NUM','HINS_U19YRS_NHI_NUM','HINS_1934YRS_NUM','HINS_1934YRS_1HI_NUM','HINS_1934YRS_2PLUS_NUM','HINS_1934YRS_NHI_NUM','HINS_3564YRS_NUM','HINS_3564YRS_1HI_NUM','HINS_3564YRS_2PLUS_NUM','HINS_3564YRS_NHI_NUM','HINS_65UPYRS_NUM','HINS_65UPYRS_1HI_NUM','HINS_65UPYRS_2PLUS_NUM','HINS_65UPYRS_NHI_NUM',
                   'HINS_NOTINSURED_PCT','HINS_NOTINSURED_19U_PCT','HINS_NOTINSURED_1934YRS_PCT','HINS_NOTINSURED_3564YRS_PCT','HINS_NOTINSURED_65UPYRS_PCT']]


df_list['Health Insurance'] = df_hins

df_dict['HINS_TOTPOP_NUM'] = 'Total'
df_dict['HINS_U19YRS_NUM'] = 'Population Under 19 years'
df_dict['HINS_U19YRS_1HI_NUM'] = 'Under 19 years - With one type of health insurance coverage'
df_dict['HINS_U19YRS_2PLUS_NUM'] = 'Under 19 years - With two or more types of health insurance coverage'
df_dict['HINS_U19YRS_NHI_NUM'] = 'Under 19 years - No health insurance coverage'
df_dict['HINS_1934YRS_NUM'] = 'Population 19 to 34 years'
df_dict['HINS_1934YRS_1HI_NUM'] = 'Population 19 to 34 years - With one type of health insurance coverage'
df_dict['HINS_1934YRS_2PLUS_NUM'] = 'Population 19 to 34 years - With two or more types of health insurance coverage'
df_dict['HINS_1934YRS_NHI_NUM'] = 'Population 19 to 34 years - No health insurance coverage'
df_dict['HINS_3564YRS_NUM'] = 'Population 35 to 64 years'
df_dict['HINS_3564YRS_1HI_NUM'] = 'Population 35 to 64 years - With one type of health insurance coverage'
df_dict['HINS_3564YRS_2PLUS_NUM'] = 'Population 35 to 64 years - With two or more types of health insurance coverage'
df_dict['HINS_3564YRS_NHI_NUM'] = 'Population 35 to 64 years - No health insurance coverage'
df_dict['HINS_65UPYRS_NUM'] = 'Population 65 years and over'
df_dict['HINS_65UPYRS_1HI_NUM'] = 'Population 65 years and over - With one type of health insurance coverage'
df_dict['HINS_65UPYRS_2PLUS_NUM'] = 'Population 65 years and over - With two or more types of health insurance coverage'
df_dict['HINS_65UPYRS_NHI_NUM'] = 'Population 65 years and over - No health insurance coverage'
df_dict['HINS_NOTINSURED_PCT'] = 'PCT OF TOTAL POPULATION THAT IS NOT INSURED'
df_dict['HINS_NOTINSURED_19U_PCT'] = 'PCT OF POPULATION <19YO THAT IS NOT INSURED'
df_dict['HINS_NOTINSURED_1934YRS_PCT'] = 'PCT OF POPULATION 19-34YO THAT IS NOT INSURED'
df_dict['HINS_NOTINSURED_3564YRS_PCT'] = 'PCT OF POPULATION 35-64YO THAT IS NOT INSURED'
df_dict['HINS_NOTINSURED_65UPYRS_PCT'] = 'PCT OF POPULATION 65YOUP THAT IS NOT INSURED'


# Industry & Occupation
* ID: ACSDT5Y2022.C24050
* Title: Industry by Occupation for the Civilian Employed Population 16 Years and Over
* Universe: Civilian employed population 16 years and over
* Source: U.S. Census Bureau, 2018-2022 American Community Survey 5-Year Estimates

In [20]:
df_ioc = pd.read_csv(import_base+'/Industry and Occupation/ACSDT5Y2022.C24050_2024-06-16T210135/ACSDT5Y2022.C24050-Data.csv',
                     dtype={'GEO_ID': str})
df_ioc['GEO_ID'] = df_ioc['GEO_ID'].str[-5:]
df_ioc = df_ioc.drop(index=0)
df_ioc = df_ioc[~df_ioc['GEO_ID'].str.startswith('72')]

df_ioc.rename(columns={'GEO_ID':'Geo_FIPS',
                        'C24050_001E':'TOT_16YRUP_EMPLOYED_NUM', #	TOT_16YRUP_EMPLOYED
                        'C24050_002E':'IOC_I_AGR_22_NUM', #	Industry: Agriculture, forestry, fishing and hunting, and mining
                        'C24050_003E':'IOC_I_CON_22_NUM', #	Industry: Construction
                        'C24050_004E':'IOC_I_MAN_22_NUM', #	Industry: Manufacturing
                        'C24050_005E':'IOC_I_TRD_22_NUM', #	Industry: Wholesale trade
                        'C24050_006E':'IOC_I_RET_22_NUM', #	Industry: Retail trade
                        'C24050_007E':'IOC_I_TRA_22_NUM', #	Industry: Transportation and warehousing, and utilities
                        'C24050_008E':'IOC_I_INF_22_NUM', #	Industry: Information
                        'C24050_009E':'IOC_I_FIN_22_NUM', #	Industry: Finance and insurance, and real estate, and rental and leasing
                        'C24050_010E':'IOC_I_PRO_22_NUM', #	Industry: Professional, scientific, and management, and administrative, and waste management services
                        'C24050_011E':'IOC_I_EDU_22_NUM', #	Industry: Educational services, and health care and social assistance
                        'C24050_012E':'IOC_I_ART_22_NUM', #	Industry: Arts, entertainment, and recreation, and accommodation and food services
                        'C24050_013E':'IOC_I_OTH_22_NUM', #	Industry: Other services, except public administration
                        'C24050_014E':'IOC_I_PUB_22_NUM', #	Industry: Public administration
                        'C24050_015E':'IOC_O_MAN_22_NUM', #	Occupations: Management, business, science, and arts
                        'C24050_029E':'IOC_O_SER_22_NUM', #	Occupations: Service
                        'C24050_043E':'IOC_O_SAO_22_NUM', #	Occupations: Sales and office
                        'C24050_057E':'IOC_O_NAT_22_NUM', #	Occupations: Natural resources, construction, and maintenance
                        'C24050_071E':'IOC_O_PRO_22_NUM' #	Occupations: Production, transportation, and material moving
                        }, inplace=True)
df_ioc['TOT_16YRUP_EMPLOYED_NUM'] = df_ioc['TOT_16YRUP_EMPLOYED_NUM'].astype(int)
df_ioc['IOC_I_AGR_22_PCT'] = df_ioc['IOC_I_AGR_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Agriculture, forestry, fishing and hunting, and mining
df_ioc['IOC_I_CON_22_PCT'] = df_ioc['IOC_I_CON_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Construction
df_ioc['IOC_I_MAN_22_PCT'] = df_ioc['IOC_I_MAN_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Manufacturing
df_ioc['IOC_I_TRD_22_PCT'] = df_ioc['IOC_I_TRD_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Wholesale trade
df_ioc['IOC_I_RET_22_PCT'] = df_ioc['IOC_I_RET_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Retail trade
df_ioc['IOC_I_TRA_22_PCT'] = df_ioc['IOC_I_TRA_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Transportation and warehousing, and utilities
df_ioc['IOC_I_INF_22_PCT'] = df_ioc['IOC_I_INF_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Information
df_ioc['IOC_I_FIN_22_PCT'] = df_ioc['IOC_I_FIN_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Finance and insurance, and real estate, and rental and leasing
df_ioc['IOC_I_PRO_22_PCT'] = df_ioc['IOC_I_PRO_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Professional, scientific, and management, and administrative, and waste management services
df_ioc['IOC_I_EDU_22_PCT'] = df_ioc['IOC_I_EDU_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Educational services, and health care and social assistance
df_ioc['IOC_I_ART_22_PCT'] = df_ioc['IOC_I_ART_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Arts, entertainment, and recreation, and accommodation and food services
df_ioc['IOC_I_OTH_22_PCT'] = df_ioc['IOC_I_OTH_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Other services, except public administration
df_ioc['IOC_I_PUB_22_PCT'] = df_ioc['IOC_I_PUB_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Industry: Public administration
df_ioc['IOC_O_MAN_22_PCT'] = df_ioc['IOC_O_MAN_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Occupations: Management, business, science, and arts
df_ioc['IOC_O_SER_22_PCT'] = df_ioc['IOC_O_SER_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Occupations: Service
df_ioc['IOC_O_SAO_22_PCT'] = df_ioc['IOC_O_SAO_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Occupations: Sales and office
df_ioc['IOC_O_NAT_22_PCT'] = df_ioc['IOC_O_NAT_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Occupations: Natural resources, construction, and maintenance
df_ioc['IOC_O_PRO_22_PCT'] = df_ioc['IOC_O_PRO_22_NUM'].astype(int) / df_ioc['TOT_16YRUP_EMPLOYED_NUM'] #	Occupations: Production, transportation, and material moving

df_ioc = df_ioc[['Geo_FIPS','TOT_16YRUP_EMPLOYED_NUM',
                 'IOC_I_AGR_22_PCT','IOC_I_CON_22_PCT','IOC_I_MAN_22_PCT','IOC_I_TRD_22_PCT','IOC_I_RET_22_PCT','IOC_I_TRA_22_PCT','IOC_I_INF_22_PCT','IOC_I_FIN_22_PCT','IOC_I_PRO_22_PCT','IOC_I_EDU_22_PCT','IOC_I_ART_22_PCT','IOC_I_OTH_22_PCT','IOC_I_PUB_22_PCT',
                 'IOC_O_MAN_22_PCT','IOC_O_SER_22_PCT','IOC_O_SAO_22_PCT','IOC_O_NAT_22_PCT','IOC_O_PRO_22_PCT']]

df_list['Industry and Occupation'] = df_ioc

df_dict['TOT_16YRUP_EMPLOYED_NUM'] = 'TOT_16YRUP_EMPLOYED'
df_dict['IOC_I_AGR_22_PCT'] = 'Percent of employed on - Industry: Agriculture, forestry, fishing and hunting, and mining'
df_dict['IOC_I_CON_22_PCT'] = 'Percent of employed on - Industry: Construction'
df_dict['IOC_I_MAN_22_PCT'] = 'Percent of employed on - Industry: Manufacturing'
df_dict['IOC_I_TRD_22_PCT'] = 'Percent of employed on - Industry: Wholesale trade'
df_dict['IOC_I_RET_22_PCT'] = 'Percent of employed on - Industry: Retail trade'
df_dict['IOC_I_TRA_22_PCT'] = 'Percent of employed on - Industry: Transportation and warehousing, and utilities'
df_dict['IOC_I_INF_22_PCT'] = 'Percent of employed on - Industry: Information'
df_dict['IOC_I_FIN_22_PCT'] = 'Percent of employed on - Industry: Finance and insurance, and real estate, and rental and leasing'
df_dict['IOC_I_PRO_22_PCT'] = 'Percent of employed on - Industry: Professional, scientific, and management, and administrative, and waste management services'
df_dict['IOC_I_EDU_22_PCT'] = 'Percent of employed on - Industry: Educational services, and health care and social assistance'
df_dict['IOC_I_ART_22_PCT'] = 'Percent of employed on - Industry: Arts, entertainment, and recreation, and accommodation and food services'
df_dict['IOC_I_OTH_22_PCT'] = 'Percent of employed on - Industry: Other services, except public administration'
df_dict['IOC_I_PUB_22_PCT'] = 'Percent of employed on - Industry: Public administration'
df_dict['IOC_O_MAN_22_PCT'] = 'Percent of employed on - Occupations: Management, business, science, and arts'
df_dict['IOC_O_SER_22_PCT'] = 'Percent of employed on - Occupations: Service'
df_dict['IOC_O_SAO_22_PCT'] = 'Percent of employed on - Occupations: Sales and office'
df_dict['IOC_O_NAT_22_PCT'] = 'Percent of employed on - Occupations: Natural resources, construction, and maintenance'
df_dict['IOC_O_PRO_22_PCT'] = 'Percent of employed on - Occupations: Production, transportation, and material moving'

# Selected Social Characteristics
* ID: ACSDP5Y2022.DP02
* Title: Selected Social Characteristics in the United States

In [21]:
df_ssc = pd.read_csv(import_base+'/Social Characteristics/ACSDP5Y2022.DP02_2024-06-16T204719/ACSDP5Y2022.DP02-Data.csv', dtype={'GEO_ID': str},low_memory=False)
df_ssc['GEO_ID'] = df_ssc['GEO_ID'].str[-5:]
df_ssc = df_ssc[~df_ssc['GEO_ID'].str.startswith('72')]
df_ssc.rename(columns={'GEO_ID':'Geo_FIPS'}, inplace=True)
df_ssc = df_ssc.drop(index=0)

# PCT UNMARRIED WOMEN 15-50 THAT GAVE BIRTH P12M
df_ssc['SSC_UNM_WOMEN_BIRTHP12M_PCT'] = df_ssc['DP02_0038E'].astype(int) / df_ssc['DP02_0037E'].astype(int)
#DP02_0037E	Number of women 15 to 50 years old who had a birth in the past 12 months
#DP02_0038E	Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)

# PCT OF PEOPLE THAT DONT LIVE IN THE SAME HOUSE AS 1YA
df_ssc['SSC_SAME_HOUSE_YA_PCT'] = df_ssc['DP02_0080E'].astype(int) / (df_ssc['DP02_0081E'].astype(int)+df_ssc['DP02_0080E'].astype(int))
#DP02_0080E	RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Same house
#DP02_0081E	RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)

# PCT OF PEOPLE THAT DONT SPEAK ENGLISH VERY WELL
df_ssc['SSC_ENG_NOT_WELL_PCT'] = (df_ssc['DP02_0115E'].astype(int) + df_ssc['DP02_0117E'].astype(int) + df_ssc['DP02_0119E'].astype(int) + df_ssc['DP02_0121E'].astype(int) + df_ssc['DP02_0123E'].astype(int)) / df_ssc['DP02_0112E'].astype(int)
#DP02_0112E	LANGUAGE SPOKEN AT HOME!!Population 5 years and over
#DP02_0115E	LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Language other than English!!Speak English less than very well""
#DP02_0117E	LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Spanish!!Speak English less than very well""
#DP02_0119E	LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Other Indo-European languages!!Speak English less than very well""
#DP02_0121E	LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Asian and Pacific Islander languages!!Speak English less than very well""
#DP02_0123E	LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Other languages!!Speak English less than very well""

# PCT OF HH WITH A COMPUTER
df_ssc['SSC_HH_COMPUTER_PCT'] = df_ssc['DP02_0153E'].astype(int) / df_ssc['DP02_0152E'].astype(int)

# PCT OF HH WITH BROADBAND
df_ssc['SSC_HH_BROADBAND_PCT'] = df_ssc['DP02_0154E'].astype(int) / df_ssc['DP02_0152E'].astype(int)
#DP02_0152E	Estimate!!COMPUTERS AND INTERNET USE!!Total households
#DP02_0153E	Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer
#DP02_0154E	Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription



df_ssc['SSC_MARITAL_15PLUS_NUM'] = df_ssc['DP02_0025E'].astype(int) + df_ssc['DP02_0031E'].astype(int)
df_ssc['SSC_MARITAL_NVM_PCT'] = (df_ssc['DP02_0026E'].astype(int) + df_ssc['DP02_0032E'].astype(int)) / df_ssc['SSC_MARITAL_15PLUS_NUM']
df_ssc['SSC_MARITAL_MARRIED_PCT'] = (df_ssc['DP02_0027E'].astype(int) + df_ssc['DP02_0033E'].astype(int)) / df_ssc['SSC_MARITAL_15PLUS_NUM']
df_ssc['SSC_MARITAL_SEPARATED_PCT'] = (df_ssc['DP02_0028E'].astype(int) + df_ssc['DP02_0034E'].astype(int)) / df_ssc['SSC_MARITAL_15PLUS_NUM']
df_ssc['SSC_MARITAL_WIDOWED_PCT'] = (df_ssc['DP02_0029E'].astype(int) + df_ssc['DP02_0035E'].astype(int)) / df_ssc['SSC_MARITAL_15PLUS_NUM']
df_ssc['SSC_MARITAL_DIVORCED_PCT'] = (df_ssc['DP02_0030E'].astype(int) + df_ssc['DP02_0036E'].astype(int)) / df_ssc['SSC_MARITAL_15PLUS_NUM']


df_ssc = df_ssc[['Geo_FIPS','SSC_UNM_WOMEN_BIRTHP12M_PCT','SSC_SAME_HOUSE_YA_PCT','SSC_ENG_NOT_WELL_PCT','SSC_HH_COMPUTER_PCT','SSC_HH_BROADBAND_PCT',
                 'SSC_MARITAL_15PLUS_NUM','SSC_MARITAL_NVM_PCT','SSC_MARITAL_MARRIED_PCT','SSC_MARITAL_SEPARATED_PCT','SSC_MARITAL_WIDOWED_PCT','SSC_MARITAL_DIVORCED_PCT']]

df_list['Social Characteristics'] = df_ssc

df_dict['SSC_UNM_WOMEN_BIRTHP12M_PCT'] = 'PCT UNMARRIED WOMEN 15-50 THAT GAVE BIRTH P12M'
df_dict['SSC_SAME_HOUSE_YA_PCT'] = 'PCT OF PEOPLE THAT DONT LIVE IN THE SAME HOUSE AS 1YA'
df_dict['SSC_ENG_NOT_WELL_PCT'] = 'PCT OF PEOPLE THAT DONT SPEAK ENGLISH VERY WELL'
df_dict['SSC_HH_COMPUTER_PCT'] = 'PCT OF HH WITH A COMPUTER'
df_dict['SSC_HH_BROADBAND_PCT'] = 'PCT OF HH WITH BROADBAND'
df_dict['SSC_MARITAL_15PLUS_NUM'] = 'TOTAL POPULATION WITH MORE THAN 15 YEARS'
df_dict['SSC_MARITAL_NVM_PCT'] =  'PCT OF 15YO+ NEVER MARRIED'
df_dict['SSC_MARITAL_MARRIED_PCT'] = 'PCT OF 15YO+ MARRIED NOT SEPARATED'
df_dict['SSC_MARITAL_SEPARATED_PCT'] =  'PCT OF 15YO+ SEPARATED'
df_dict['SSC_MARITAL_WIDOWED_PCT'] =  'PCT OF 15YO+ WIDOWED'
df_dict['SSC_MARITAL_DIVORCED_PCT'] =  'PCT OF 15YO+ DIVORCED'


# Selected Economic Characteristics
* ID: ACSDP5Y2022.DP03
* Title: Selected Economic Characteristics

In [22]:
df_sec = pd.read_csv(import_base+'/Economic Characteristics/ACSDP5Y2022.DP03-Data.csv', dtype={'GEO_ID': str},low_memory=False)
df_sec['GEO_ID'] = df_sec['GEO_ID'].str[-5:]
df_sec = df_sec[~df_sec['GEO_ID'].str.startswith('72')]
df_sec = df_sec.drop(index=0)
df_sec = df_sec.replace('-', np.nan)

# New columns dictionary
new_columns = {
    'Geo_FIPS': df_sec['GEO_ID'].str[-5:],
    'NI_POP_NUM': df_sec['DP03_0095PE'].astype('int'),
    'NI_INS_PCT': round(df_sec['DP03_0096PE'].astype('float')/100,4),
    'NI_INS_PRI_PCT': round(df_sec['DP03_0097PE'].astype('float')/100,4),
    'NI_INS_PUB_PCT': round(df_sec['DP03_0098PE'].astype('float')/100,4),
    'NI_NOT_INS_PCT': round(df_sec['DP03_0099PE'].astype('float')/100,4),
    'NI_19-64_POP_NUM': df_sec['DP03_0102PE'].astype('int'),
    'NI_19-64_LAB_POP_NUM': df_sec['DP03_0103PE'].astype('int'),
    'NI_19-64_LAB_EMP_POP_NUM': df_sec['DP03_0104PE'].astype('int'),
    'NI_19-64_LAB_EMP_INS_PCT': round(df_sec['DP03_0105PE'].astype('float')/100,4),
    'NI_19-64_LAB_EMP_INS_PRI_PCT': round(df_sec['DP03_0106PE'].astype('float')/100,4),
    'NI_19-64_LAB_EMP_INS_PUB_PCT': round(df_sec['DP03_0107PE'].astype('float')/100,4),
    'NI_19-64_LAB_EMP_NOT_INS_PCT': round(df_sec['DP03_0108PE'].astype('float')/100,4),
    'NI_19-64_LAB_UNE_POP_NUM': df_sec['DP03_0109PE'].astype('int'),
    'NI_19-64_LAB_UNE_INS_PCT': round(df_sec['DP03_0110PE'].astype('float')/100,4),
    'NI_19-64_LAB_UNE_INS_PRI_PCT': round(df_sec['DP03_0111PE'].astype('float')/100,4),
    'NI_19-64_LAB_UNE_INS_PUB_PCT': round(df_sec['DP03_0112PE'].astype('float')/100,4),
    'NI_19-64_LAB_UNE_NOT_INS_PCT': round(df_sec['DP03_0113PE'].astype('float')/100,4),
    'NI_19-64_NLAB_POP_NUM': df_sec['DP03_0114PE'].astype('int'),
    'NI_19-64_NLAB_INS_PCT': round(df_sec['DP03_0115PE'].astype('float')/100,4),
    'NI_19-64_NLAB_INS_PRI_PCT': round(df_sec['DP03_0116PE'].astype('float')/100,4),
    'NI_19-64_NLAB_INS_PUB_PCT': round(df_sec['DP03_0117PE'].astype('float')/100,4),
    'NI_19-64_NLAB_NOT_INS_PCT': round(df_sec['DP03_0118PE'].astype('float')/100,4)
}

# Convert dictionary to DataFrame
df_sec = pd.DataFrame(new_columns)


# Create a new DataFrame to de-fragment
df_sec = df_sec.copy()


df_list['Economic Characteristics'] = df_sec

df_dict['NI_POP_NUM'] = 'POP Civilian noninstitutionalized population'
df_dict['NI_INS_PCT'] = 'PCT Civilian noninstitutionalized population!!With health insurance coverage'
df_dict['NI_INS_PRI_PCT'] = 'PCT Civilian noninstitutionalized population!!With health insurance coverage!!With private health insurance'
df_dict['NI_INS_PUB_PCT'] = 'PCT Civilian noninstitutionalized population!!With health insurance coverage!!With public coverage'
df_dict['NI_NOT_INS_PCT'] = 'PCT Civilian noninstitutionalized population!!No health insurance coverage'

df_dict['NI_19-64_POP_NUM'] = 'POP Civilian noninstitutionalized population 19 to 64 years'
df_dict['NI_19-64_LAB_POP_NUM'] = 'POP Civilian noninstitutionalized population 19 to 64 years!!In labor force:'
df_dict['NI_19-64_LAB_EMP_POP_NUM'] = 'POP Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Employed:'
df_dict['NI_19-64_LAB_EMP_INS_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Employed:!!With health insurance coverage'
df_dict['NI_19-64_LAB_EMP_INS_PRI_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Employed:!!With health insurance coverage!!With private health insurance'
df_dict['NI_19-64_LAB_EMP_INS_PUB_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Employed:!!With health insurance coverage!!With public coverage'
df_dict['NI_19-64_LAB_EMP_NOT_INS_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Employed:!!No health insurance coverage'

df_dict['NI_19-64_LAB_UNE_POP_NUM'] = 'POP Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Unemployed'
df_dict['NI_19-64_LAB_UNE_INS_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Unemployed:!!With health insurance coverage'
df_dict['NI_19-64_LAB_UNE_INS_PRI_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Unemployed:!!With health insurance coverage!!With private health insurance'
df_dict['NI_19-64_LAB_UNE_INS_PUB_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Unemployed:!!With health insurance coverage!!With public coverage'
df_dict['NI_19-64_LAB_UNE_NOT_INS_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!In labor force:!!Unemployed:!!No health insurance coverage'

df_dict['NI_19-64_NLAB_POP_NUM'] = 'POP Civilian noninstitutionalized population 19 to 64 years!!Not in labor force'
df_dict['NI_19-64_NLAB_INS_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!Not in labor force:!!With health insurance coverage'
df_dict['NI_19-64_NLAB_INS_PRI_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!Not in labor force:!!With health insurance coverage!!With private health insurance'
df_dict['NI_19-64_NLAB_INS_PUB_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!Not in labor force:!!With health insurance coverage!!With public coverage'
df_dict['NI_19-64_NLAB_NOT_INS_PCT'] = 'PCT Civilian noninstitutionalized population 19 to 64 years!!Not in labor force:!!No health insurance coverage'


# Area Health Resources - Workforce
* https://data.hrsa.gov/topics/health-workforce/ahrf









In [23]:
df_ahr = pd.read_csv(import_base+'/Health - Area Health Resources Files/DATA/ahrf2023.csv', dtype={'fips_st_cnty': str},
                     encoding="ISO-8859-1", low_memory=False)
df_ahr = df_ahr[~df_ahr['fips_st_cnty'].str.startswith('72')]

df_ahr.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

new_columns = {
    'Geo_FIPS': df_ahr['fips_st_cnty'].astype('str'),
    'HCPWF_ALLMDS_NUM': df_ahr['md_nf_fed_21'].astype('int'),
    'HCPWF_ACTIVEDO_NUM': df_ahr['do_nf_fed_activ_21'].astype('int'),
    'HCPWF_ACTIVEMD_NUM': df_ahr['md_nf_fed_activ_21'].astype('int'),
    'HCPWF_ALLPHYSICIANS_NUM': df_ahr['do_nf_fed_activ_21'].astype('int') + df_ahr['md_nf_fed_activ_21'].astype('int'),
    'HCPWF_NURSES_NUM': df_ahr['aprn_npi_22'].astype('int'),
    'HCPWF_DENTISTS_NUM': df_ahr['dent_nf_fed_proflly_activ_21'].astype('int'),
    'HCPWF_PHYSASSIST_NUM': df_ahr['pa_npi_22'].astype('int'),
    'HCPWF_PHYSPER1000_PCT': (df_ahr['do_nf_fed_activ_21'].astype('int') + df_ahr['md_nf_fed_activ_21'].astype('int')) / df_ahr['popn_est_21'].astype(int) * 1000,
    'HCPWF_NURSESPER1000_PCT': df_ahr['aprn_npi_22'].astype('int') / df_ahr['popn_est_22'].astype(int) * 1000,
    'HCPWF_DENTSPER1000_PCT': df_ahr['dent_nf_fed_proflly_activ_21'].astype('int') / df_ahr['popn_est_21'].astype(int) * 1000,
}


# Convert dictionary to DataFrame
df_ahr = pd.DataFrame(new_columns)

# Create a new DataFrame to de-fragment
df_ahr = df_ahr.copy()

df_list['Area Health Resources - Workforce'] = df_ahr

df_dict['HCPWF_ALLMDS_NUM'] = 'M.D., All M.D. (Active and inactive) Inactive Status includes physicians who are retired, semiretired, working part-time, temporarily not in practice, or not active for other reasons and indicated they worked 20 hours or less per week. '
df_dict['HCPWF_ACTIVEDO_NUM'] = 'Includes D.O. with active status, employed by federal government or non-federal industry. Inactive Status includes physicians who are retired, semiretired, working part time, temporarily not in practice, or not active for other reasons and indicated they worked 20 hours or less per week. Not Classified Status includes physicians who did not provide information on their type of practice or their present employment. Federal Status is defined as full time employment by the federal government, including the Army, Navy, Air Force, Veteran‘s Administration, the Public Health Service and other federally funded agencies. M.D.'
df_dict['HCPWF_ACTIVEMD_NUM'] = 'Active M.D. employed by federal government or non-federal industry.'
df_dict['HCPWF_ALLPHYSICIANS_NUM'] = 'All active Physicians (MD + DO)'
df_dict['HCPWF_NURSES_NUM'] = 'Includes Advanced Practice Registered Nurses with a National Provider Identifier (NPI) only. APRN includes advanced practice midwife, certified registered nurse anesthetist, clinical nurse specialist, and nurse practitioner. '
df_dict['HCPWF_DENTISTS_NUM'] = 'Includes Dentists with professionally active status.'
df_dict['HCPWF_PHYSASSIST_NUM'] = 'Includes Physician Assistants with a National Provider Identifier (NPI) only.'
df_dict['HCPWF_PHYSPER1000_PCT'] = 'All active Physicians per 1000 people in the county (2021)'
df_dict['HCPWF_NURSESPER1000_PCT'] = 'Nurses per 1000 people in the county (2022)'
df_dict['HCPWF_DENTSPER1000_PCT'] = 'Dentists per 1000 people in the county (2021)'


<ipython-input-23-2d3c3cb877ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ahr.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# Drug Poisoning
* https://data.cdc.gov/NCHS/NCHS-Drug-Poisoning-Mortality-by-County-United-Sta/pbkm-d27e/about_data
* https://www.cdc.gov/nchs/data-visualization/drug-poisoning-mortality/
* https://www.cdc.gov/nchs/nvss/deaths.htm?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fnchs%2Fdeaths.htm

In [24]:
df_drug = pd.read_csv(import_base+'/Health - Drug Poisoning/NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv', dtype={'FIPS': str})
df_drug = df_drug[df_drug['Year'] == 2020]
df_drug = df_drug[['FIPS','Model-based Death Rate']]
df_drug.rename(columns={'FIPS': 'Geo_FIPS','Model-based Death Rate':'DRUG_MORTALITY_RATE_PCT'}, inplace=True)
df_drug['Geo_FIPS'] = df_drug['Geo_FIPS'].str.replace(' ', '')
df_drug["Geo_FIPS"] = df_drug["Geo_FIPS"].astype(str)
df_drug['Geo_FIPS'] = df_drug['Geo_FIPS'].apply(lambda x: x.zfill(5))
df_drug['DRUG_MORTALITY_RATE_PCT'] = df_drug['DRUG_MORTALITY_RATE_PCT'].apply(lambda x: round(x/100,4))

df_list['Drug Poisoning Mortality'] = df_drug

df_dict['DRUG_MORTALITY_RATE_PCT'] = 'Model-based Death Rate for 2020'

In [25]:
df_drug

,Geo_FIPS,DRUG_MORTALITY_RATE_PCT
11,01001,0.1431
32,01003,0.2814
40,01005,0.0765
74,01007,0.2902
78,01009,0.3112
...,...,...
59506,56037,0.2929
59525,56039,0.1313
59544,56041,0.3342
59563,56043,0.2175


# Geographic Characteristics
* https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html


In [26]:

df_geo = pd.read_table(import_base + '/Geographic Characteristics/2023_Gaz_counties_national.txt',
                   dtype={
                       'GEOID': str,
                       'NAME': str,
                       'ALAND': int,
                       'AWATER': int,
                       'ALAND_SQMI': float,
                       'AWATER_SQMI': float,
                       'INTPTLAT': float})

df_geo.rename(columns={'GEOID': 'Geo_FIPS',
                       'ALAND_SQMI': 'ALAND_SQMI_NUM',
                       'AWATER_SQMI': 'AWATER_SQMI_NUM',
                       'INTPTLAT': 'GEOLAT_NUM'}, inplace=True)
df_geo.rename(columns={df_geo.columns[-1]: 'GEOLONG_NUM'}, inplace=True)
df_geo = df_geo[['Geo_FIPS','ALAND_SQMI_NUM','AWATER_SQMI_NUM','GEOLAT_NUM','GEOLONG_NUM']]

df_list['Geographic Characteristics'] = df_geo

df_dict['ALAND_SQMI_NUM'] = 'Land Area from U.S. Gazetteer Files 2023 in Square Meters (Land Area (square miles) - Created for statistical purposes only)'
df_dict['AWATER_SQMI_NUM'] = 'Water Area'
df_dict['GEOLAT_NUM'] = 'Latitude'
df_dict['GEOLONG_NUM'] = 'Longitude'

df_geo

,Geo_FIPS,ALAND_SQMI_NUM,AWATER_SQMI_NUM,GEOLAT_NUM,GEOLONG_NUM
0,01001,594.455,9.914,32.532237,-86.646440
1,01003,1589.863,437.410,30.659218,-87.746067
2,01005,885.008,19.507,31.870253,-85.405104
3,01007,622.470,3.696,33.015893,-87.127148
4,01009,644.891,5.738,33.977358,-86.566440
...,...,...,...,...,...
3217,72145,45.848,22.327,18.455128,-66.397883
3218,72147,50.788,213.203,18.125418,-65.432474
3219,72149,35.637,1.399,18.130718,-66.472244
3220,72151,55.215,28.028,18.059858,-65.859871


# Housing Characteristics

*  ACSDP5Y2022.DP04



In [27]:
df_housing = pd.read_csv(import_base+'/Housing Characteristics/ACSDP5Y2022.DP04-Data.csv', dtype={'GEO_ID': str},low_memory=False)
df_housing['GEO_ID'] = df_housing['GEO_ID'].str[-5:]
df_housing = df_housing[~df_housing['GEO_ID'].str.startswith('72')]
df_housing = df_housing.drop(index=0)
df_housing = df_housing.replace('-', np.nan)


# New columns dictionary
new_columns = {
    'Geo_FIPS': df_housing['GEO_ID'].str[-5:],
    'HC_UNITS_TOTAL_NUM': df_housing['DP04_0001E'].astype('int'),
    'HC_UNITS_OCCU_NUM': df_housing['DP04_0002E'].astype('int'),
    'HC_OWNER_OCCU_NUM': df_housing['DP04_0046E'].astype('int'),
    'HC_OWNER_OCCU_HHSIZE_NUM': round(df_housing['DP04_0048E'].astype('float'),2), #---  Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of owner-occupied unit
    'HC_OWNER_OCCU_W_MORT_NUM': df_housing['DP04_0091E'].astype('int'), #--- 	Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units with a mortgage
    'HC_OWNER_OCCU_NO_MORT_NUM': df_housing['DP04_0092E'].astype('int'), #--- 	Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units without a mortgage
    'HC_RENTER_OCCU_NUM': df_housing['DP04_0047E'].astype('int'), #--- 	Estimate!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
    'HC_RENTER_OCCU_PAYING_RENT_NUM': df_housing['DP04_0126E'].astype('int'), #--- 	Estimate!!GROSS RENT!!Occupied units paying rent
    'HC_RENTER_OCCU_HHSIZE_NUM': round(df_housing['DP04_0049E'].astype('float'),2), #---  Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit
    'HC_OCCU_VEHICLES_O_NUM': df_housing['DP04_0058E'].astype('int'), #--- 	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available
    'HC_OCCU_VEHICLES_1_NUM': df_housing['DP04_0059E'].astype('int'), #--- 	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available
    'HC_OCCU_VEHICLES_2_NUM': df_housing['DP04_0060E'].astype('int'), #--- 	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available
    'HC_OCCU_VEHICLES_3PLUS_NUM': df_housing['DP04_0061E'].astype('int'), #--- 	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available
    'HC_NO_PLUMBING_NUM': df_housing['DP04_0073E'].astype('int'), #--- 	Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities
    'HC_NO_KITCHEN_NUM': df_housing['DP04_0074E'].astype('int'), #--- 	Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete kitchen facilities
    'HC_RENTER_OCCU_PCT': round(df_housing['DP04_0047PE'].astype('float')/100,4), #---	Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
    'HC_OWNER_OCCU_PCT': round(df_housing['DP04_0046PE'].astype('float')/100,4), # Percent!!HOUSING TENURE!!Occupied housing units!!Owner-occupied
    'HC_NO_BEDROOMS_PCT': round(df_housing['DP04_0039PE'].astype('float')/100,4), #	Percent!!BEDROOMS!!Total housing units!!No bedroom
    'HC_1_BEDROOMS_PCT': round(df_housing['DP04_0040PE'].astype('float')/100,4),	# Percent!!BEDROOMS!!Total housing units!!1 bedroom
    'HC_2_BEDROOMS_PCT': round(df_housing['DP04_0041PE'].astype('float')/100,4),	# Percent!!BEDROOMS!!Total housing units!!2 bedrooms
    'HC_3_BEDROOMS_PCT': round(df_housing['DP04_0042PE'].astype('float')/100,4),	# Percent!!BEDROOMS!!Total housing units!!3 bedrooms
    'HC_4_BEDROOMS_PCT': round(df_housing['DP04_0043PE'].astype('float')/100,4),	# Percent!!BEDROOMS!!Total housing units!!4 bedrooms
    'HC_5PLUS_BEDROOMS_PCT': round(df_housing['DP04_0044PE'].astype('float')/100,4), #	Percent!!BEDROOMS!!Total housing units!!5 or more bedrooms
    'HC_OWNER_OCCU_W_MORT_PCT': round(df_housing['DP04_0091PE'].astype('float')/100,4), #	Percent!!MORTGAGE STATUS!!Owner-occupied units!!Housing units with a mortgage
    'HC_OWNER_OCCU_W_MORT_SMOCAPI_20LESS_PCT': round(df_housing['DP04_0111PE'].astype('float')/100,4), # Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!Less than 20.0 percent
    'HC_OWNER_OCCU_W_MORT_SMOCAPI_20TO24_PCT': round(df_housing['DP04_0112PE'].astype('float')/100,4), # Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!20.0 to 24.9 percent
    'HC_OWNER_OCCU_W_MORT_SMOCAPI_25TO29_PCT': round(df_housing['DP04_0113PE'].astype('float')/100,4), # Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!25.0 to 29.9 percent
    'HC_OWNER_OCCU_W_MORT_SMOCAPI_30TO34_PCT': round(df_housing['DP04_0114PE'].astype('float')/100,4), # Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!30.0 to 34.9 percent
    'HC_OWNER_OCCU_W_MORT_SMOCAPI_35PLUS_PCT': round(df_housing['DP04_0115PE'].astype('float')/100,4), # Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!35.0 percent or more
    #'HC_OWNER_OCCU_W_MORT_SMOCAPI_NOTCOMPUTED_PCT': round(df_housing['DP04_0116PE'].astype('float')/100,4), # Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!Not computed
    'HC_OWNER_OCCU_NO_MORT_PCT': round(df_housing['DP04_0117PE'].astype('float')/100,4), #	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)
    'HC_OCCU_VEHICLES_O_PCT': round(df_housing['DP04_0058PE'].astype('float')/100,4), #	Percent!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available
    'HC_OCCU_VEHICLES_1_PCT': round(df_housing['DP04_0059PE'].astype('float')/100,4), #	Percent!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available
    'HC_OCCU_VEHICLES_2_PCT': round(df_housing['DP04_0060PE'].astype('float')/100,4), #	Percent!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available
    'HC_OCCU_VEHICLES_3PLUS_PCT': round(df_housing['DP04_0061PE'].astype('float')/100,4), #	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available
    'HC_NO_PLUMBING_PCT': round(df_housing['DP04_0073PE'].astype('float')/100,4), # Percent!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities
    'HC_NO_KITCHEN_PCT': round(df_housing['DP04_0074PE'].astype('float')/100,4), # Percent!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete kitchen facilities
    'HC_OCPROOM_1ORLESS_PCT': round(df_housing['DP04_0077PE'].astype('float')/100,4), #	Percent!!OCCUPANTS PER ROOM!!Occupied housing units!!1.00 or less
    'HC_OCPROOM_1POINT5_PCT': round(df_housing['DP04_0078PE'].astype('float')/100,4), #	Percent!!OCCUPANTS PER ROOM!!Occupied housing units!!1.01 to 1.50
    'HC_OCPROOM_1POINT5PLUS_PCT': round(df_housing['DP04_0079PE'].astype('float')/100,4)
    }

# Convert dictionary to DataFrame
df_housing = pd.DataFrame(new_columns)

# Create a new DataFrame to de-fragment
df_housing = df_housing.copy()

df_housing['HC_OCCU_VEHICLES_123PLUS_NUM'] = df_housing['HC_OCCU_VEHICLES_1_NUM'] + df_housing['HC_OCCU_VEHICLES_2_NUM'] + df_housing['HC_OCCU_VEHICLES_3PLUS_NUM']
df_housing['HC_OCCU_VEHICLES_123PLUS_PCT'] = round((df_housing['HC_OCCU_VEHICLES_1_PCT'] + df_housing['HC_OCCU_VEHICLES_2_PCT'] + df_housing['HC_OCCU_VEHICLES_3PLUS_PCT']).astype(float)/100,4)


df_list['Housing Characteristics'] = df_housing

df_dict['HC_UNITS_TOTAL_NUM'] = 'Total housing units DP04_0001E'
df_dict['HC_UNITS_OCCU_NUM'] = 'Occupied housing units DP04_0002E'
df_dict['HC_OWNER_OCCU_UNITS_NUM'] = 'Occupied housing units - Owner-occupied DP04_0046E'
df_dict['HC_OWNER_OCCU_HHSIZE_NUM'] = 'Occupied housing units - Average household size of owner-occupied unit DP04_0048E'
df_dict['HC_RENTER_OCCU_NUM'] = 'Occupied housing units - Renter-occupied DP04_0047E'
df_dict['HC_RENTER_OCCU_PAYING_RENT_NUM'] = 'Occupied housing units - Renter-occupied DP04_0126E'
df_dict['HC_RENTER_OCCU_HHSIZE_NUM'] = 'DP04_0049E Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit'
df_dict['HC_OCCU_VEHICLES_O_NUM'] = 'DP04_0058E	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available'
df_dict['HC_OCCU_VEHICLES_1_NUM'] = 'DP04_0059E	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available'
df_dict['HC_OCCU_VEHICLES_2_NUM'] = 'DP04_0060E	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available'
df_dict['HC_OCCU_VEHICLES_3PLUS_NUM'] = 'DP04_0061E	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available'
df_dict['HC_OCCU_VEHICLES_123PLUS_NUM'] = ''
df_dict['HC_NO_PLUMBING_NUM'] = 'DP04_0073E	Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities'
df_dict['HC_NO_KITCHEN_NUM'] = 'DP04_0074E	Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete kitchen facilities'
df_dict['HC_RENTER_OCCU_NUM'] = 'DP04_0126E	Estimate!!GROSS RENT!!Occupied units paying rent'
df_dict['HC_OWNER_OCCU_NUM'] = 'DP04_0090E	Estimate!!MORTGAGE STATUS!!Owner-occupied units'
df_dict['HC_OWNER_OCCU_W_MORT_NUM'] = 'DP04_0091E	Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units with a mortgage'
df_dict['HC_OWNER_OCCU_NO_MORT_NUM'] = 'DP04_0092E	Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units without a mortgage'
df_dict['HC_RENTER_OCCU_PCT'] = 'DP04_0047PE	Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied'
df_dict['HC_OWNER_OCCU_PCT'] = 'DP04_0046PE	Percent!!HOUSING TENURE!!Occupied housing units!!Owner-occupied'
df_dict['HC_NO_BEDROOMS_PCT'] = 'DP04_0039PE	Percent!!BEDROOMS!!Total housing units!!No bedroom'
df_dict['HC_1_BEDROOMS_PCT'] = 'DP04_0040PE	Percent!!BEDROOMS!!Total housing units!!1 bedroom'
df_dict['HC_2_BEDROOMS_PCT'] = 'DP04_0041PE	Percent!!BEDROOMS!!Total housing units!!2 bedrooms'
df_dict['HC_3_BEDROOMS_PCT'] = 'DP04_0042PE	Percent!!BEDROOMS!!Total housing units!!3 bedrooms'
df_dict['HC_4_BEDROOMS_PCT'] = 'DP04_0043PE	Percent!!BEDROOMS!!Total housing units!!4 bedrooms'
df_dict['HC_5PLUS_BEDROOMS_PCT'] = 'DP04_0044PE	Percent!!BEDROOMS!!Total housing units!!5 or more bedrooms'
df_dict['HC_OWNER_OCCU_W_MORT_PCT'] = 'DP04_0091PE	Percent!!MORTGAGE STATUS!!Owner-occupied units!!Housing units with a mortgage'
df_dict['HC_OWNER_OCCU_W_MORT_SMOCAPI_20LESS_PCT'] = 'DP04_0111PE	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!Less than 20.0 percent'
df_dict['HC_OWNER_OCCU_W_MORT_SMOCAPI_20TO24_PCT'] = 'DP04_0112PE	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!20.0 to 24.9 percent'
df_dict['HC_OWNER_OCCU_W_MORT_SMOCAPI_25TO29_PCT'] = 'DP04_0113PE	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!25.0 to 29.9 percent'
df_dict['HC_OWNER_OCCU_W_MORT_SMOCAPI_30TO34_PCT'] = 'DP04_0114PE	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!30.0 to 34.9 percent'
df_dict['HC_OWNER_OCCU_W_MORT_SMOCAPI_35PLUS_PCT'] = 'DP04_0115PE	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!35.0 percent or more'
df_dict['HC_OWNER_OCCU_W_MORT_SMOCAPI_NOTCOMPUTED_PCT'] = 'DP04_0116PE	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!Not computed'
df_dict['HC_OWNER_OCCU_NO_MORT_PCT'] = 'DP04_0117PE	Percent!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)'
df_dict['HC_OCCU_VEHICLES_O_PCT'] = 'DP04_0058PE	Percent!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available'
df_dict['HC_OCCU_VEHICLES_1_PCT'] = 'DP04_0059PE	Percent!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available'
df_dict['HC_OCCU_VEHICLES_2_PCT'] = 'DP04_0060PE	Percent!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available'
df_dict['HC_OCCU_VEHICLES_3PLUS_PCT'] = 'DP04_0061PE	Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available'
df_dict['HC_OCCU_VEHICLES_123PLUS_PCT'] = ''
df_dict['HC_NO_PLUMBING_PCT'] = 'DP04_0073PE	Percent!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities'
df_dict['HC_NO_KITCHEN_PCT'] = 'DP04_0074PE	Percent!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete kitchen facilities'
df_dict['HC_OCPROOM_1ORLESS_PCT'] = 'DP04_0077PE	Percent!!OCCUPANTS PER ROOM!!Occupied housing units!!1.00 or less'
df_dict['HC_OCPROOM_1POINT5_PCT'] = 'DP04_0078PE	Percent!!OCCUPANTS PER ROOM!!Occupied housing units!!1.01 to 1.50'
df_dict['HC_OCPROOM_1POINT5PLUS_PCT'] = 'DP04_0079PE	Percent!!OCCUPANTS PER ROOM!!Occupied housing units!!1.51 or more'

df_housing

,Geo_FIPS,HC_UNITS_TOTAL_NUM,HC_UNITS_OCCU_NUM,HC_OWNER_OCCU_NUM,HC_OWNER_OCCU_HHSIZE_NUM,HC_OWNER_OCCU_W_MORT_NUM,HC_OWNER_OCCU_NO_MORT_NUM,HC_RENTER_OCCU_NUM,HC_RENTER_OCCU_PAYING_RENT_NUM,HC_RENTER_OCCU_HHSIZE_NUM,HC_OCCU_VEHICLES_O_NUM,HC_OCCU_VEHICLES_1_NUM,HC_OCCU_VEHICLES_2_NUM,HC_OCCU_VEHICLES_3PLUS_NUM,HC_NO_PLUMBING_NUM,HC_NO_KITCHEN_NUM,HC_RENTER_OCCU_PCT,HC_OWNER_OCCU_PCT,HC_NO_BEDROOMS_PCT,HC_1_BEDROOMS_PCT,HC_2_BEDROOMS_PCT,HC_3_BEDROOMS_PCT,HC_4_BEDROOMS_PCT,HC_5PLUS_BEDROOMS_PCT,HC_OWNER_OCCU_W_MORT_PCT,HC_OWNER_OCCU_W_MORT_SMOCAPI_20LESS_PCT,HC_OWNER_OCCU_W_MORT_SMOCAPI_20TO24_PCT,HC_OWNER_OCCU_W_MORT_SMOCAPI_25TO29_PCT,HC_OWNER_OCCU_W_MORT_SMOCAPI_30TO34_PCT,HC_OWNER_OCCU_W_MORT_SMOCAPI_35PLUS_PCT,HC_OWNER_OCCU_NO_MORT_PCT,HC_OCCU_VEHICLES_O_PCT,HC_OCCU_VEHICLES_1_PCT,HC_OCCU_VEHICLES_2_PCT,HC_OCCU_VEHICLES_3PLUS_PCT,HC_NO_PLUMBING_PCT,HC_NO_KITCHEN_PCT,HC_OCPROOM_1ORLESS_PCT,HC_OCPROOM_1POINT5_PCT,HC_OCPROOM_1POINT5PLUS_PCT,HC_OCCU_VEHICLES_123PLUS_NUM,HC_OCCU_VEHICLES_123PLUS_PCT
1,01001,24457,22308,16832,2.70,9676,7156,5476,4826,2.34,888,7080,7741,6599,111,90,0.245,0.755,0.006,0.035,0.168,0.479,0.264,0.048,0.575,0.560,0.139,0.074,0.063,0.165,69.94,0.040,0.317,0.347,0.296,0.005,0.004,0.986,0.007,0.007,21420,0.0096
2,01003,125113,90802,70708,2.61,41421,29287,20094,17892,2.25,2056,28223,38546,21977,386,800,0.221,0.779,0.014,0.061,0.207,0.487,0.190,0.040,0.586,0.514,0.140,0.079,0.055,0.212,289.08,0.023,0.311,0.425,0.242,0.004,0.009,0.980,0.012,0.007,88746,0.0098
3,01005,11673,9016,5858,2.54,2499,3359,3158,2804,2.23,1058,3002,2887,2069,20,45,0.350,0.650,0.006,0.066,0.263,0.516,0.133,0.016,0.427,0.573,0.089,0.060,0.081,0.197,32.92,0.117,0.333,0.320,0.229,0.002,0.005,0.962,0.031,0.007,7958,0.0088
4,01007,9046,7216,5465,3.00,1994,3471,1751,1317,2.54,544,1836,2539,2297,38,132,0.243,0.757,0.009,0.045,0.252,0.538,0.125,0.031,0.365,0.552,0.141,0.044,0.111,0.153,34.50,0.075,0.254,0.352,0.318,0.005,0.018,0.987,0.009,0.004,6672,0.0092
5,01009,24677,21626,17144,2.85,8647,8497,4482,3559,2.13,1047,5375,7072,8132,96,183,0.207,0.793,0.008,0.035,0.238,0.539,0.142,0.037,0.504,0.523,0.174,0.090,0.058,0.155,83.64,0.048,0.249,0.327,0.376,0.004,0.008,0.976,0.020,0.003,20579,0.0095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,56037,19198,16335,11982,2.56,6590,5392,4353,4211,2.52,330,3865,6078,6062,35,71,0.266,0.734,0.017,0.065,0.229,0.448,0.151,0.089,0.550,0.576,0.140,0.104,0.034,0.146,52.72,0.020,0.237,0.372,0.371,0.002,0.004,0.977,0.019,0.004,16005,0.0098
3141,56039,13365,9645,6003,2.46,3654,2349,3642,3385,2.11,183,2996,4033,2433,0,85,0.378,0.622,0.054,0.114,0.260,0.341,0.163,0.069,0.609,0.424,0.146,0.093,0.100,0.237,23.49,0.019,0.311,0.418,0.252,0.000,0.009,0.940,0.031,0.029,9462,0.0098
3142,56041,8831,7586,5914,2.73,3486,2428,1672,1493,2.50,283,1933,2363,3007,22,2,0.220,0.780,0.006,0.057,0.229,0.447,0.189,0.072,0.589,0.679,0.150,0.034,0.035,0.101,24.10,0.037,0.255,0.311,0.396,0.003,0.000,0.963,0.032,0.005,7303,0.0096
3143,56043,3856,3414,2446,2.38,1184,1262,968,798,1.80,161,853,1234,1166,0,0,0.284,0.716,0.003,0.070,0.344,0.296,0.223,0.066,0.484,0.378,0.227,0.146,0.090,0.160,12.38,0.047,0.250,0.361,0.342,0.000,0.000,0.979,0.013,0.008,3253,0.0095


# Merging all datasets!

In [28]:
# prompt: iterate through df_list keys
df_reframe = df_popcounts['Geo_FIPS']

for key in df_list.keys():
   df_reframe = pd.merge(df_reframe, df_list[key], on="Geo_FIPS", how="outer")
   print(key + " - added: " + str(df_reframe.shape))


# ! REMOVE ALL ROWS WITH CONTINUUM CODE EMPTY #
df_reframe = df_reframe[~df_reframe["R_CONTINUUM_CODE"].isna()]
print(df_reframe.shape)

Rural Continuum with Race and Ethnicity - added: (3144, 17)
Economic Type - added: (3156, 20)
Economic Data - added: (3205, 210)
Unemployment - added: (3283, 233)
Education - added: (3285, 237)
Elections - added: (3323, 249)
Poverty - added: (3323, 253)
Food Access - added: (3323, 256)
COVID-19 Vaccination - added: (3327, 258)
Health Measures - added: (3328, 268)
Population Changes - added: (3328, 306)
GDP - added: (3335, 313)
GINI - added: (3335, 314)
Health Insurance - added: (3335, 336)
Industry and Occupation - added: (3335, 355)
Social Characteristics - added: (3335, 366)
Economic Characteristics - added: (3335, 388)
Area Health Resources - Workforce - added: (3335, 398)
Drug Poisoning Mortality - added: (3335, 399)
Geographic Characteristics - added: (3335, 403)
Housing Characteristics - added: (3335, 444)
(3144, 444)


In [29]:
now = datetime.datetime.now(datetime.timezone(-datetime.timedelta(hours=5)))
date_time = str(now.year) + str(now.month).zfill(2) + str(now.day).zfill(2) + "T" + str(now.hour).zfill(2) + str(now.minute).zfill(2) + str(now.second).zfill(2)

version_name = 'V24'
df_reframe.to_csv('/content/drive/MyDrive/FCB/Rural Reframe/Exports/'+version_name+'_Dataset-'+ date_time +'.csv')

print(version_name+'-'+ date_time +'.csv')

data_dictionary = [{'Column Name':'Geo_FIPS',
                    'Description':'[KEY] Federal Information Processing Standard (FIPS) code for the geographic area.',
                   'Type':'str',
                   'Source':'N/A'}]

for key in df_list.keys():
    for column in df_list[key].columns:
        if column in df_reframe and column != 'Geo_FIPS':
            data_dictionary.append({
                'Column Name': column,
                'Description': df_dict[column],
                'Type': type(df_reframe[column][0]).__name__,
                'Source': key
            })

data_dictionary = pd.DataFrame(data_dictionary)

data_dictionary.reset_index(drop=True, inplace=True)

data_dictionary.to_csv('/content/drive/MyDrive/FCB/Rural Reframe/Exports/'+version_name+'_Dataset_META.csv')

V24-20240709T143243.csv
